In [4]:
#!/bin/python3
#

from pathlib import Path
import shutil
import datetime
from datetime import datetime
import time
import sys
import cx_Oracle
from datetime import date, timedelta
import os
global wlog_txt, wdir_permissao, wsem_erro, wtempo_inicial, wtempo_final, wcount_arquivos
global wtsn_rept, wtem_reg_002, wdir_permissao, linha, wlista_arquivo, wsair, nome_txt, wid_001, wid_002, wid_003, wid_004, \
wid_005, wid_020, wid_021, wid_022, wid_023, wid_100, wid_principal, wcarro, wdata_arquivo, wnrinterno_002_ant, \
wvalor_debitado_ant, wtsn_002_ant, wusado, wid_principal_res, wempresa_res, wdata_res, wnr_carro_res, wmot_tsp_res, \
wmot_cod_prodata_res, wmot_matricula_res, wcob_tsp_res, wcob_cod_prodata_res, wcob_matricula_res, wqtd_pax_total_res, \
wqtd_pax_grat_res, wqtd_pax_pagt_res, wqtd_pax_botao_res, wqtd_pax_vt_res, wqtd_pax_bu_res, wrec_bruta_res, wrec_botao_res, \
wrec_vt_res, wrec_bu_bruto_res, wrec_bu_receb_res, wrec_subsidio_res, wcod_empresa, wnr_linha, wnome_linha, wnome_empresa, \
wturno_princ, wmot_ant, wcob_ant, wnome_arquivo_db, wid_001_db, wcartoes, dir_leitura, arquivos, wdata_util, wdesc_emissor, \
wdesc_aplicacao


In [5]:
def mes_extenso(data):
    meses = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
    data = data[:3] + meses[int(data[3:5]) - 1]+ data[5:]
    return data


def atualiza_data(data, hora):
    hora = hora[:2]
    list_hora = ['00', '01', '02']
    if(hora in list_hora): 
        curr_date_temp = datetime.strptime(data, "%d/%m/%Y")
        new_date = curr_date_temp - timedelta(days=1)
        format_date = new_date.strftime("%d/%b/%Y")
        return format_date
    else:
        curr_date_temp = datetime.strptime(data, "%d/%m/%Y")
        new_date = curr_date_temp.strftime("%d-%b-%Y")
        return new_date

    
def formata_data(data):
    curr_date_temp = datetime.strptime(data, "%d/%m/%Y")
    new_date = curr_date_temp.strftime("%d-%b-%Y")
    return new_date


def atualiza_data_dir(data, hora):
    hora = hora[:2]
    list_hora = ['00', '01', '02', '03']
    if(hora in list_hora): 
        curr_date_temp = datetime.strptime(data, "%d/%m/%Y")
        new_date = curr_date_temp - timedelta(days=1)
        return new_date
    else:
        new_date = datetime.strptime(data, "%d/%m/%Y")
        return new_date   
     

def converte_juliana(data):
    curr_date = "31/12/2002"
    curr_date_temp = datetime.strptime(curr_date, "%d/%m/%Y")
    new_date = curr_date_temp + timedelta(days=int(data))
    format_date = new_date.strftime("%d-%b-%Y")
    return format_date

def converte_juliana_bil002(data):
    curr_date = "31/12/2002"
    curr_date_temp = datetime.strptime(curr_date, "%d/%m/%Y")
    new_date = curr_date_temp + timedelta(days=int(data))
    format_date = new_date.strftime("%d/%m/%Y")
    return format_date


def convert_hora(segundos): 
    return time.strftime("%H:%M:%S", time.gmtime(segundos)) 

def prox_sequencia():
    cursor = conn.cursor()
    querystring =  "SELECT S_ID_BILHETAGEM.NEXTVAL FROM DUAL"
    cursor.execute(querystring)
    w_id_principal = cursor.fetchone()
    cursor.close()
    for i in w_id_principal:
        w_id_principal = i
    return w_id_principal



def cartao_provisorio():
    try:
        with conn.cursor() as cursor:
            sql_query = """SELECT TRAF_NR_CARTAO_CRACHA_V3LAN AS CARTAO,
                                  TRAF_FUNC_MATRIC_CRACHA_V3LAN as MATRICULA 
                           FROM TRAF_CRACHA_V3LAN
                           WHERE TRAF_DT_FIM_CRACHA_V3LAN is null      
                        """
            cursor.execute(sql_query)
            dados = dict(cursor.fetchall())
            
    except Exception as err:
        print(err)
        pass
    return dados


def busca_linha(id_linha, id_dados):
    wdados ={206:['1330101','133002','SANTA SOFIA X CENTRAL'],
             208:['1330101','133004','LAGOINHA X CENTRAL'],
             210:['1330101','133006','SANTA SOFIA X CASTELO C/AR'],
             211:['1330101','133007','ITAGUAI X NITEROI'],
             214:['1330101','133009','CAMPO GRANDE X ILHA DA MADEIRA'],
             216:['1330101','133010','PARACAMBI X VILA GENI'],
             212:['1330101','133011','CAMPO GRANDE X PARACAMBI'],
             213:['1330101','133012','SANTA SOFIA X CAMPO GRANDE'],
             215:['1330101','133013','CAMPO GRANDE X PONTE COBERTA'],
             619:['1330101','133014','PARACANBI X VILA GENI DIRETA'],
             620:['1330101','133015','PONTE COBERTA X C. GRANDE DIRETA'],
             218:['1330101','133016','CAMPO GRANDE X CAMPO LINDO'],
             217:['1330101','133020','CACARIA  X  ITAGUAI'],
             219:['1330101','133021','CAMPO GRANDE X PRACA CASTILHO'],
             221:['1330101','133023','JARDIM MARACANA X SEROPEDICA'],
             569:['1330101','133024','CABUCU  X   COELHO NETO'],
             701:['1280101','128039','SAO JOAO X BARRA DA TIJUCA'],
             205:['1330101','133061','ITAGUAI X CENTRAL C/AR'],
             207:['1330101','133062','Seropedica x Central C/AR'],
             203:['1330101','133063','Cabucu x Central C/AR'],
             195:['0070101','007013','Jardim Redentor x Belford Roxo'],
             196:['0070101','007014','Jardim Redentor x Belford Roxo'],
             197:['0070101','007015','JARDIM REDENTOR X BELFORD ROXO'],
             198:['0070101','007016','Jardim Redentor x Belford Roxo'],
             199:['0070101','007017','JARDIM REDENTOR X BELFORD ROXO'],
             220:['1330101','133026','ITAGUAI   X  MAL. HERMES'],
             163:['1280101','128011','Sargento Roncale x N. Iguacu'],
             156:['1280101','128004','N.Iguacu x D. de Caxias'],
             175:['1280101','128025','Pavuna x J. Bom Pastor'],
             209:['1330101','133005','ITAGUAI X CASTELO C/AR'],
             310:['1710101','171001','SAO JOAO DE MERITI X CAXIAS'],
             300:['1670101','167001','VILAR DOS TELES  X CASTELO'],
             303:['1670101','167002','PRACA DA BANDEIRA  X CENTRAL'],
             307:['1670101','167003','EDEM X CASTELO'],
             299:['1670101','167004','EDEM X CENTRAL'],
             305:['1670101','167005','P. ARARUAMA   X   CENTRAL'],
             306:['1670101','167006','VENDA VELHA   X    CENTRAL'],
             308:['1670101','167007','EDEM   X    CENTRAL'],
             304:['1670101','167008','SAO JOAO    X     CENTRAL'],
             302:['1670101','167009','V. TIRADENTES  X  CENTRAL'],
             309:['1710101','171002','SAO JOAO X CAXIAS'],
             512:['1330101','133027','SANTA SOFIA X COELHO NETO'],
             513:['1330101','133028','ITAGUAI X COELHO NETO'],
             153:['1280101','128001','Sao J.de Meriti x D. de Caxias'],
             154:['1280101','128002','Sao J. de Meriti x D.de Caxias'],
             155:['1280101','128003','Sao J. de Meriti x D.de Caxias'],
             157:['1280101','128005','Jardim Botanico x Caxias'],
             158:['1280101','128006','Jardim do Ipe x Caxias'],
             159:['1280101','128007','Sao Vicente x Caxias'],
             160:['1280101','128008','D.de Caxias x Nova Iguacu'],
             161:['1280101','128009','D. de Caxias x Nova Iguacu'],
             162:['1280101','128010','Lote Xv x Nova Iguacu'],
             164:['1280101','128012','Parque Sao Jose x N. Iguacu'],
             243:['1280101','128013','Nova Iguacu x Xerem'],
             165:['1280101','128014','Santa Marta x Cascadura'],
             166:['1280101','128015','Jardim Metropole x Cascadura'],
             167:['1280101','128016','Jardim  Botanico x Cascadura'],
             168:['1280101','128017','Vila Norma x Cascadura'],
             169:['1280101','128018','Cosmorama x Cascadura'],
             170:['1280101','128019','Eden x Cascadura'],
             171:['1280101','128020','Sao Joao x Cascadura'],
             172:['1280101','128021','Novo Rio x Cascadura'],
             173:['1280101','128022','Shopping Sendas x Madureira'],
             244:['1280101','128023','Sao Vicente x Meier'],
             174:['1280101','128024','Pavuna x Vale do Ipe'],
             176:['1280101','128026','Coelho da Rocha x Castelo'],
             177:['1280101','128027','Pavuna x Santa Tereza'],
             178:['1280101','128028','Nilopolis x Pavuna'],
             179:['1280101','128029','Nilopolis x Pavuna'],
             180:['1280101','128030','Nilopolis x SHOPPING G RIO'],
             181:['1280101','128031','Coelho da Rocha x Praca Xv'],
             182:['1280101','128032','Coelho da Rocha x Central'],
             183:['1280101','128033','Praca da Bandeira x Central'],
             184:['1280101','128034','Coelho Branco x Central'],
             185:['1280101','128036','Pavuna x Sargento Roncale'],
             186:['1280101','128037','Pavuna x Sargento Roncale'],
             187:['1280101','128038','Nova Aurora x Madureira'],
             679:['1280101','128041','CAXIAS X CAMPO GRANDE'],
             684:['1330101','133038','CHAPERO X SANTA CRUZ'],
             682:['1330101','133036','ITAGUAI X CESARAO'],
             683:['1330101','133037','ITAGUAI X SANTA CRUZ'],
             685:['1330101','133039','CAMPO GRANDE X ITAGUAI'],
             686:['1330101','133040','CAMPO GRANDE X ITAGUAI'],
             716:['1670101','167014','PRACA BANDEIRA  X CENTRAL'],
             717:['1670101','167016','BOM PASTOR  X  CENTRAL'],
             713:['0070101','007018','JARDIM NOIA  X  SHOPPING GRANDE RIO'],
             719:['0070101','007019','NOVA IGUACU X JARDIM TROPICAL'],
             188:['0070101','7001','Edem X Araruama'],
             189:['0070101','007002','Sao Joao X Vilar dos Teles'],
             190:['0070101','007003','SAO JOAO X VILAR DOS TELES'],
             191:['0070101','007004','SAO JOAO X MATADOURO'],
             192:['0070101','007005','SAO JOAO X GRANDE RIO'],
             193:['0070101','007007','SAO JOAO X ARARUAMA'],
             194:['0070101','007011','SAO JOAO X BACIA'],
             612:['0040101','004050','RESERVAS'],
             204:['1330101','133001','ITAGUAI X CENTRAL'],
             202:['1330101','133003','CABUCU X CENTRAL'],
             611:['1330101','133035','CACARIA X VILA GENI'],
             668:['1330101','133029','CACARIA  X  SEROPEDICA'],
             670:['1330101','133030','ITAGUAI X SANTA SOFIA'],
             618:['1280101','128040','Parque Sao Jose X Nova Iguaçu'],
             671:['1330101','133032','CAMPO LINDO  X PARACAMBI'],
             669:['1330101','133031','CACARIA  X SEROPEDICA'],
             677:['1670101','167011','COELHO DA ROCHA  X CASTELO'],
             678:['1670101','167013','EDEM X CASTELO'],
             700:['1280101','128042','DUQUE DE CAXIAS X ITAGUAI'],
             715:['1710101','171003','SAO JOAO X COPACABANA']}
     
    if not id_linha in wdados:
        if id_dados == 0:
            return 999999
        elif id_dados == 1:
            return 999999
        else:
            return 'NAO CADASTRADA'
    else:    
        return wdados[id_linha][id_dados]


    
def desc_emissor(id_emissor):
    
    wemissores = { 1:'FETRANSPOR',
                   2:'RIO ONIBUS',
                   3:'NITEROI',
                   4:'NOVA IGUACU',
                   5:'DUQUE DE CAXIAS',
                   7:'NAO CADASTRADO',
                  15:'NAO CADASTRADO',
                  99:'NAO CADASTRADO'
                 }
    
    if not id_emissor in wemissores:
        return 'NAO CADASTRADO'
    else:    
        return wemissores[id_emissor]


def desc_aplicacao(id_emissor, id_aplicacao):
    
    if (id_emissor == 1):
        wapp_emissor1 = { 100:'CARTAO EXPRESSO',
                          400:'VT FETRANSPOR',
                          411:'VT INTEGRACAO 1',
                          412:'VT INTEGRACAO 2',
                          413:'VT INTEGRACAO 3',
                          414:'VT INTEGRACAO 5',
                          442:'VT PAN2007',
                          450:'VT RAPIDO',
                         1023:'CARD DATA 2 FETRANSPOR'
                        }
        
        if not id_aplicacao in wapp_emissor1:
            return 'NAO CADASTRADO'
        else:    
            return wapp_emissor1[id_aplicacao]
        
    elif(id_emissor == 2):
        wapp_emissor2 = {   0:'CARD DATA 1 GENERAL',
                            1:'CARD DATA 3 INDEX',
                            2:'CARD DATA 0 SIGNATURE',
                          100:'BOTOEIRA',
                          220:'CARD DATA 1 GENERAL (OBSOLETE)',
                          400:'VT FETRANSPOR/FUNCIONAL',
                          500:'VT 1 RIO ONIBUS',
                          510:'VT 2 RIO ONIBUS',
                          600:'ESPECIAL',
                          610:'ESPECIAL C/AC',
                          611:'ACOMPANHANTE',
                          700:'RODOVIARIO RIO ONIBUS',
                          701:'ROD CIDADAO RIO ONIBUS',
                          702:'JOVEM APRENDIZ',
                          704:'FISCALIZACAO',
                          710:'FUNCIONAL',
                          711:'FUNCIONAL APRENDIZ',
                          720:'CLAUSULA 15',
                          800:'SENIOR RIO ONIBUS',
                          900:'EST MUN RIO ONIBUS',
                          910:'EST EST RIO ONIBUS',
                          920:'EST FED RIO ONIBUS',
                         1000:'OPERADOR',
                         1001:'OPERADOR FUNCIONAL',
                         1007:'SERVICE',
                         1008:'DATA COLLECT',
                         1009:'MULA MAIN',
                         1010:'MULA APP',
                         1011:'DATA COLLECT - STATISTICAL PRICES',
                         1023:'CARD DATA 2 - INTEGRATION'
                        }
        
        if not id_aplicacao in wapp_emissor2:
            return 'NAO CADASTRADO'
        else:    
            return wapp_emissor2[id_aplicacao]
        
    elif(id_emissor == 3):
        wapp_emissor3 = {   0:'CARD DATA 1 GENERAL',
                            1:'CARD DATA 3 INDEX',
                            2:'CARD DATA 0 SIGNATURE',
                          100:'BOTOEIRA',
                          222:'CARD DATA 1 GENERAL (OBSOLETE)',
                          400:'VT NITEROI/FETRANSPOR',
                          411:'VT INTEGRACAO 1',
                          500:'VT 1 SETRERJ',
                          510:'VT 2 SETRERJ',
                          600:'ESPECIAL',
                          610:'ESPECIAL C/AC',
                          611:'ACOMPANHANTE',
                          700:'RODOVIARIO SETRERJ',
                          701:'ROD CIDADAO SETRERJ',
                          704:'FISCALIZACAO',
                          710:'FUNCIONAL',
                          720:'CLAUSULA 15',
                          800:'SENIOR SETRERJ',
                          900:'ESTUDANTE MUNICIPAL',
                          910:'ESTUDANTE ESTADUAL',
                          920:'ESTUDANTE FEDERAL',
                          930:'DOENTES CRONICOS',
                          940:'DOENTE CRONICO COM ACOMPANHANTE',
                         1000:'OPERADOR',
                         1001:'OPERADOR FUNCIONAL',
                         1007:'SERVICE',
                         1008:'DATA COLLECT',
                         1011:'DATA COLLECT STATISTICAL PRICES',
                         1023:'CD2 NIT'
                        }
        
        if not id_aplicacao in wapp_emissor3:
            return 'NAO CADASTRADO'
        else:    
            return wapp_emissor3[id_aplicacao]

    elif(id_emissor == 4):
        wapp_emissor4 = { 100:'BOTOEIRA',
                          700:'RODOVIARIO SETRANSPANI',
                          701:'ROD CIDADAO SETRANSPANI',
                          703:'RODOVIARIO FRETAMENTO',
                          705:'RODOV FRET CIDADAO',
                          710:'FUNCIONAL',
                          800:'SENIOR SETRANSPANI',
                          910:'EST EST SETRANSPANI',
                         1000:'OPERADOR',
                         1023:'NI'
                        }
        
        if not id_aplicacao in wapp_emissor4:
            return 'NAO CADASTRADO'
        else:    
            return wapp_emissor4[id_aplicacao]

    elif(id_emissor == 5):
        wapp_emissor5 = { 700:'RODOVIARIO SETRANSDUC',
                          701:'ROD CIDADAO SETRANSDUC',
                          704:'FISCALIZACAO',
                          710:'FUNCIONAL',
                          800:'SENIOR SETRANSDUC',
                         1000:'OPERADOR',
                         1001:'OPERADOR FUNCIONAL',
                         1023:'DC'
                        }
        
        if not id_aplicacao in wapp_emissor5:
            return 'NAO CADASTRADO'
        else:    
            return wapp_emissor5[id_aplicacao]
        
    elif(id_emissor == 7):
        wapp_emissor7 = { 800:'SENIOR SETRANSOL',
                         1023:'CARD DATA 2 INTEGRATION'
                        }
        
        if not id_aplicacao in wapp_emissor7:
            return 'NAO CADASTRADO'
        else:    
            return wapp_emissor7[id_aplicacao]

    elif(id_emissor == 15):
        wapp_emissor15 = { 612:'ESPECIAL INTERMUNICIPAL',
                           613:'ESPECIAL INTERMUNICIPAL C/AC',
                           614:'ACOMPANHANTE INTERMUNIPAL',
                           900:'EST MUN INTERMUNICIPAL',
                           910:'EST EST INTERMUNICIPAL',
                           920:'EST FED INTERMUNICIPAL',
                           950:'ESCOLAR OI',
                           951:'EST MUN INTERMUNICIPAL',
                          1023:'CARD DATA 2 INTEGRATION'
                         }
        
        if not id_aplicacao in wapp_emissor15:
            return 'NAO CADASTRADO'
        else:    
            return wapp_emissor15[id_aplicacao]

    elif(id_emissor == 99):
        wapp_emissor99 = { 2:'ACOMPANHANTE'
                         }
        
        if not id_aplicacao in wapp_emissor99:
            return 'NAO CADASTRADO'
        else:    
            return wapp_emissor99[id_aplicacao]
    
    
    
def insert_resumo_carro(ID_PRINCIPAL, EMPRESA, DATA, NR_CARRO, MOT_TSP, MOT_COD_PRODATA, MOT_MATRICULA, COB_TSP, 
                        COB_COD_PRODATA, COB_MATRICULA, QTD_PAX_TOTAL, QTD_PAX_GRAT, QTD_PAX_PAGT, QTD_PAX_BOTAO, 
                        QTD_PAX_VT, QTD_PAX_BU, REC_BRUTA, REC_BOTAO, REC_VT, 
                        REC_BU_BRUTO, REC_BU_RECEB, REC_SUBSIDIO):
    try:
        cursor = conn.cursor()
        sql_insert = """INSERT INTO BIL_RESUMO_CARRO (ID_PRINCIPAL, EMPRESA, DATA, NR_CARRO, MOT_TSP, MOT_COD_PRODATA, 
                                                   MOT_MATRICULA, COB_TSP, COB_COD_PRODATA, COB_MATRICULA, QTD_PAX_TOTAL,
                                                   QTD_PAX_GRAT, QTD_PAX_PAGT, QTD_PAX_BOTAO, QTD_PAX_VT, QTD_PAX_BU, 
                                                   REC_BRUTA, REC_BOTAO, REC_VT, REC_BU_BRUTO, REC_BU_RECEB, REC_SUBSIDIO) 
                       VALUES(:1, :2, :3, :4, :5, :6, :7, :8, :9, :10, :11, :12, :13, :14, :15, :16, :17, :18, :19, :20, 
                              :21, :22)"""
        cursor.execute(sql_insert, (ID_PRINCIPAL, EMPRESA, DATA, NR_CARRO, MOT_TSP, MOT_COD_PRODATA, MOT_MATRICULA, COB_TSP, 
                                    COB_COD_PRODATA, COB_MATRICULA, QTD_PAX_TOTAL, QTD_PAX_GRAT, QTD_PAX_PAGT, QTD_PAX_BOTAO, 
                                    QTD_PAX_VT, QTD_PAX_BU, REC_BRUTA, REC_BOTAO, REC_VT, REC_BU_BRUTO, REC_BU_RECEB,
                                    REC_SUBSIDIO))  
     
    except Exception as err:
        #wsair = True
        #wtexto = 'Erro ao tentar na funcao insert_resumo_carro  =  '+str(err)
        #insert_bil_erro(nome_txt, linha, wtexto)
        print(err)
        wsem_erro = True
        with open(Path(wdir_permissao / wlog_txt), 'a') as notepad3:
            notepad3.write('Erro no insert_resumo_carro\n')
            notepad3.write('NOME_ARQ:'+str(nome_txt)+'\nID PRINCIPAL: '+str(ID_PRINCIPAL)+'\nCARRO: '+str(NR_CARRO)+
                            '\nDATA: '+str(DATA)+'\nMOT: '+str(MOT_MATRICULA)+'\nCOB: '+str(COB_MATRICULA))
            notepad3.write('\n'+ str(err) +'\n')
            notepad3.write('\n=============================================================\n')
                
        pass
    else:    
        conn.commit()
        cursor.close() 
        
        
def insert_resumo_empresa(EMPRESA, DATA, PAX_TOTAL, PAX_BOTAO, PAX_VT, PAX_BU,
                          PAX_GRATUIDADE, RECEITA_BRUTA, RECEITA_BOTAO, RECEITA_VT, 
                          RECEITA_BU_BRUTO, RECEITA_BU_RECEBIDO, RECEITA_SUBSIDIO, 
                          QTD_TOTAL_GUIAS):   
    try:
        with conn.cursor() as cursor:
            sql_query = """SELECT COUNT(*) FROM BIL_RESUMO_EMPRESA
                            WHERE EMPRESA = :WEMPRESA
                              AND DATA = :WDATA       
                        """
            cursor.execute(sql_query, WEMPRESA=EMPRESA, WDATA=DATA)
            rows = cursor.fetchall()
            if rows == 0:
                 with conn.cursor() as cursor:
                    try:
                        cursor = conn.cursor()
                        sql_insert = """INSERT INTO BIL_RESUMO_EMPRESA (EMPRESA, DATA, PAX_TOTAL, PAX_BOTAO, PAX_VT, PAX_BU,
                                                                        PAX_GRATUIDADE, RECEITA_BRUTA, RECEITA_BOTAO, RECEITA_VT, 
                                                                        RECEITA_BU_BRUTO, RECEITA_BU_RECEBIDO, RECEITA_SUBSIDIO, 
                                                                        QTD_TOTAL_GUIAS) 
                                            VALUES(:1, :2, :3, :4, :5, :6, :7, :8, :9, :10, :11, :12, :13, :14)"""
                        cursor.execute(sql_insert, (EMPRESA, DATA, PAX_TOTAL, PAX_BOTAO, PAX_VT, PAX_BU,
                                                    PAX_GRATUIDADE, RECEITA_BRUTA, RECEITA_BOTAO, RECEITA_VT, 
                                                    RECEITA_BU_BRUTO, RECEITA_BU_RECEBIDO, RECEITA_SUBSIDIO, 
                                                    QTD_TOTAL_GUIAS))  
                    except Exception as err:
                        print(err)
                    else:
                        conn.commit()
                        cursor.close() 
                        
            else:
                try:
                    cursor = conn.cursor()
                    sql_insert = """UPDATE BIL_RESUMO_EMPRESA SET PAX_TOTAL = PAX_TOTAL + :1, 
                                                                  PAX_BOTAO = PAX_BOTAO + :2, 
                                                                  PAX_VT    = PAX_VT + :3, 
                                                                  PAX_BU    = PAX_BU + :4,
                                                                  PAX_GRATUIDADE = PAX_GRATUIDADE + :5,
                                                                  RECEITA_BRUTA = RECEITA_BRUTA + :6,
                                                                  RECEITA_BOTAO = RECEITA_BOTAO + :7, 
                                                                  RECEITA_VT = RECEITA_VT + :8, 
                                                                  RECEITA_BU_BRUTO    = RECEITA_BU_BRUTO + :9,  
                                                                  RECEITA_BU_RECEBIDO   = RECEITA_BU_RECEBIDO + :10,   
                                                                  RECEITA_SUBSIDIO  = RECEITA_SUBSIDIO + :11, 
                                                                  QTD_TOTAL_GUIAS = QTD_TOTAL_GUIAS + 1
                                                            WHERE EMPRESA = :12
                                                              AND DATA = :13
                     
                                        VALUES(:1, :2, :3, :4, :5, :6, :7, :8, :9, :10, :11, :12, :13)"""
                    cursor.execute(sql_insert, (PAX_TOTAL, PAX_BOTAO, PAX_VT, PAX_BU,
                                                PAX_GRATUIDADE, RECEITA_BRUTA, RECEITA_BOTAO, RECEITA_VT, 
                                                RECEITA_BU_BRUTO, RECEITA_BU_RECEBIDO, RECEITA_SUBSIDIO,
                                                EMPRESA, DATA))  
                except Exception as err:
                    print(err)
                else:
                    conn.commit()
                    cursor.close()            
    except Exception as err:
        print(err)
    else:
        conn.commit()
        cursor.close()
        
def update_resumo_carro(REC_BRUTA, REC_BOTAO, REC_VT, REC_BU_BRUTO, REC_BU_RECEB, REC_SUBSIDIO, ID_PRINCIPAL):
    try:
        cursor = conn.cursor()
        sql_update =""" UPDATE BIL_RESUMO_CARRO SET REC_BRUTA = :1, 
                                                    REC_BOTAO = :2, 
                                                    REC_VT = :3, 
                                                    REC_BU_BRUTO = :4,
                                                    REC_BU_RECEB = :5, 
                                                    REC_SUBSIDIO = :6
                                                    WHERE ID_PRINCIPAL = :7
                    """
        cursor.execute(sql_update, (REC_BRUTA, REC_BOTAO, REC_VT, REC_BU_BRUTO, REC_BU_RECEB, REC_SUBSIDIO, ID_PRINCIPAL))
        
    except Exception as err:
        print(err)
        #wtexto = 'Erro ao tentar na funcao update tns repetido =  '+str(err)
        #wtipo_erro = str(WID_PRINCIPAL)+' = '+str(WVALOR)+' = '+str(WSUBSIDIO)+' = '+str(WCARTAO)+' = '+str(WTSN)
        #insert_bil_erro(nome_txt,wtipo_erro, wtexto)
        #pass
    else:
        conn.commit()
        cursor.close()
        
        
def update_tsn_repetido(WVALOR,  WSUBSIDIO, WID_PRINCIPAL, WCARTAO, WTSN):
    try:
        cursor = conn.cursor()
        sql_update =""" UPDATE BIL_002 SET VALOR_DEBITADO = :1, 
                                           DIF_VALOR_DEBITADO = :2
                        WHERE ID_PRINCIPAL = :3
                        AND NUM_INTERNO_CARTAO = :4
                        AND TSN_CARTAO = :5
                    """
        cursor.execute(sql_update, (WVALOR, WSUBSIDIO, WID_PRINCIPAL, WCARTAO, WTSN))
        
    except Exception as err:
        wsem_erro = True
        with open(Path(wdir_permissao / wlog_txt), 'a') as notepad3:
            notepad3.write('Erro no update_tsn_repetido\n')
            notepad3.write('NOME_ARQ:'+str(nome_txt)+'\nID PRINCIPAL: '+str(WID_PRINCIPAL)+'\nVALOR: '+str(WVALOR)+
                            '\nCARTAO: '+str(WCARTAO)+'\nTSN: '+str(WTSN))
            notepad3.write('\n'+ str(err) +'\n')
            notepad3.write('\n=============================================================\n')
        wtexto = 'Erro ao tentar na funcao update tns repetido =  '+str(err)
        wtipo_erro = str(WID_PRINCIPAL)+' = '+str(WVALOR)+' = '+str(WSUBSIDIO)+' = '+str(WCARTAO)+' = '+str(WTSN)
        #insert_bil_erro(nome_txt,wtipo_erro, wtexto)
        pass
    else:      
        conn.commit()
        cursor.close()

        
def insert_principal(ID,NOME,DATA,HORA,VERSAO,CARRO,TIPO_ONI,EMPRESA,DATA_UTIL,SEQ_ARQ,DATA_MALOTE, HORA_TXT, DATA_ARQUIVO,NOME_ARQUIVO_DB):
    try:
        try:
            cursor = conn.cursor()
            sql_delete = """ DELETE FROM BIL_PRINCIPAL WHERE NOME_ARQUIVO_DB = :NOME_ARQUIVO_DB"""
            cursor.execute(sql_delete, {'NOME_ARQUIVO_DB':NOME_ARQUIVO_DB})
            conn.commit()
        except Exception as err:
            with open(Path(wdir_permissao / wlog_txt), 'a') as notepad3:
                notepad3.write('Erro no DELETE_BIL_PRINCIPAL\n')
                notepad3.write('NOME_ARQ:'+str(NOME))
                notepad3.write('\n'+ str(err) +'\n')
                notepad3.write('\n=============================================================\n')            
            print(err)
            pass
        try:    
            cursor = conn.cursor()
            sql_delete = """ DELETE TB_UDP_GARAGE WHERE NOME_ARQUIVO = :NOME_TXT"""
            cursor.execute(sql_delete, {'NOME_TXT':NOME})
            conn.commit()
        except Exception as err:
            with open(Path(wdir_permissao / wlog_txt), 'a') as notepad3:
                notepad3.write('Erro no DELETE_TB_UDP_GARAGE\n')
                notepad3.write('NOME_ARQ:'+str(NOME))
                notepad3.write('\n'+ str(err) +'\n')
                notepad3.write('\n=============================================================\n')            
            print(err)
            pass
            
        cursor = conn.cursor()
        sql_insert = """INSERT INTO BIL_PRINCIPAL (ID_PRINCIPAL, NOME_ARQUIVO, DATA_GERACAO, HORA_GERACAO,
                                                    VERSAO, CARRO, ID_TIPO_ONIBUS, ID_EMPRESA, DATA_UTIL,
                                                    SEQUENCIAL_ARQUIVO, DATA_MALOTE, HORA_TXT, DATA_ARQUIVO,NOME_ARQUIVO_DB) 
                       VALUES(:1, :2, :3, :4, :5, :6, :7, :8, :9, :10, :11, :12, :13, :14)"""
        cursor.execute(sql_insert, (ID,NOME,DATA,HORA,VERSAO,CARRO,TIPO_ONI,EMPRESA,DATA_UTIL,SEQ_ARQ,DATA_MALOTE, 
                                    HORA_TXT, DATA_ARQUIVO,NOME_ARQUIVO_DB))  
    except Exception as err:
        wsem_erro = True
        with open(Path(wdir_permissao / wlog_txt), 'a') as notepad3:
            notepad3.write('Erro no insert_principal\n')
            notepad3.write('NOME_ARQ:'+str(nome_txt)+'\nID PRINCIPAL: '+str(ID)+'\nCARRO: '+str(CARRO)+
                            '\nDATA: '+str(DATA_ARQUIVO)+'\nHORA: '+str(HORA_TXT)+'\nSEQ: '+str(SEQ_ARQ))
            notepad3.write('\n'+ str(err) +'\n')
            notepad3.write('\n=============================================================\n') 
        
        shutil.move(arquivo, (caminho_arq / Path(dir_leitura[2]) / arquivo.name))   
        
        #wsair = True
        #wtexto = 'Erro ao tentar na funcao insert_principal  =  '+str(err)
        #insert_bil_erro(NOME,HORA_TXT, wtexto)
        print(err)
        pass
    else:    
        conn.commit()
        cursor.close() 
        
def atualiza_financeiro_principal(STATUS_FINANCEIRO, ID_PRINCIPAL):
    try:
        cursor = conn.cursor()
        sql_update = """UPDATE BIL_PRINCIPAL SET STATUS_FINANCEIRO = :STATUS_FINANCEIRO 
                        WHERE ID_PRINCIPAL = :ID_PRINCIPAL"""
        cursor.execute(sql_update, {'STATUS_FINANCEIRO':STATUS_FINANCEIRO, 'ID_PRINCIPAL':ID_PRINCIPAL})
    except Exception as err:
        wsem_erro = True
        with open(Path(wdir_permissao / wlog_txt), 'a') as notepad3:
            notepad3.write('Erro no atualiza_financeiro_principal\n')
            notepad3.write('NOME_ARQ:'+str(nome_txt)+'\nID PRINCIPAL: '+str(ID_PRINCIPAL))
            notepad3.write('\n'+ str(err) +'\n')
            notepad3.write('\n=============================================================\n')        
        wtexto = 'Erro ao tentar na funcao atualiza_financeiro_principal  =  '+str(err)
        #insert_bil_erro(nome_txt,linha, wtexto)
        pass
    else:
        conn.commit()
        cursor.close()
        

def insert_bil100(ID_PRINCIPAL, ID_BIL_100, TIPO, VERSAO, TSP, NR_ONIBUS, TIPO_ONIBUS, NUMERO_CHIP_SAM, LSEQUENCE_NUMBER, 
                  ASS_REG):
    try:
        cursor = conn.cursor()
        sql_insert = """INSERT INTO BIL_100 (ID_PRINCIPAL, ID_BIL_100, TIPO, VERSAO, TSP, NR_ONIBUS, TIPO_ONIBUS, 
                                             NUMERO_CHIP_SAM, LSEQUENCE_NUMBER, ASS_REG) 
                        VALUES(:1, :2, :3, :4, :5, :6, :7, :8, :9, :10)"""
        cursor.execute(sql_insert, (ID_PRINCIPAL, ID_BIL_100, TIPO, VERSAO, TSP, NR_ONIBUS, TIPO_ONIBUS, NUMERO_CHIP_SAM, 
                                    LSEQUENCE_NUMBER, ASS_REG))  
    except Exception as err:
        wsem_erro = True
        with open(Path(wdir_permissao / wlog_txt), 'a') as notepad3:
            notepad3.write('Erro no insert_bil100\n')
            notepad3.write('NOME_ARQ:'+str(nome_txt)+'\nID PRINCIPAL: '+str(ID_PRINCIPAL)+'\nCARRO: '+str(NR_ONIBUS)+
                            '\nTIPO: '+str(TIPO)+'\nID_BIL_100: '+str(ID_BIL_100)+'\nNR_CHIP_SAM: '+str(NUMERO_CHIP_SAM))
            notepad3.write('\n'+ str(err) +'\n')
            notepad3.write('\n=============================================================\n')        
        wtexto = 'Erro ao tentar na funcao bil_100  =  '+str(err)
        #insert_bil_erro(nome_txt,linha, wtexto)
        pass
    else:        
        conn.commit()
        cursor.close()

def insert_bil001(ID_PRINCIPAL, ID_BIL_001, TIPO, VERSAO, TSP, NR_ONIBUS, ESTADO_SERVICO, DATA_ARQ, HORA_ARQ, HORA_TXT, 
                  MOT_TSP, MOT_COD_PRODATA, MOT_MATRICULA, COB_TSP, COB_COD_PRODATA, COB_MATRICULA, NR_LINHA_DETALHE, 
                  NR_LINHA_RIOCARD, CODIGO_SECAO, SENTIDO, TURNO, ASS_REG):
    try:
        cursor = conn.cursor()
        sql_insert = """INSERT INTO BIL_001 (ID_PRINCIPAL, ID_BIL_001, TIPO, VERSAO, TSP, NR_ONIBUS, ESTADO_SERVICO, DATA_ARQ,
                                             HORA_ARQ, HORA_TXT, MOT_TSP, MOT_COD_PRODATA, MOT_MATRICULA, COB_TSP, 
                                             COB_COD_PRODATA, COB_MATRICULA, NR_LINHA_DETALHE, NR_LINHA_RIOCARD, CODIGO_SECAO,
                                             SENTIDO, TURNO, ASS_REG) 
                       VALUES(:1, :2, :3, :4, :5, :6, :7, :8, :9, :10, :11, :12, :13, :14, :15, :16, :17, :18, :19, :20, 
                              :21, :22)"""
        cursor.execute(sql_insert, (ID_PRINCIPAL, ID_BIL_001, TIPO, VERSAO, TSP, NR_ONIBUS, ESTADO_SERVICO, DATA_ARQ, HORA_ARQ,
                                    HORA_TXT, MOT_TSP, MOT_COD_PRODATA, MOT_MATRICULA, COB_TSP, COB_COD_PRODATA, COB_MATRICULA, 
                                    NR_LINHA_DETALHE, NR_LINHA_RIOCARD, CODIGO_SECAO, SENTIDO, TURNO, ASS_REG))  
    except Exception as err:
        wsem_erro = True
        with open(Path(wdir_permissao / wlog_txt), 'a') as notepad3:
            notepad3.write('Erro no insert_bil001\n')
            notepad3.write('NOME_ARQ:'+str(nome_txt)+'\nID PRINCIPAL: '+str(ID_PRINCIPAL)+'\nCARRO: '+str(NR_ONIBUS)+
                            '\nHORA: '+str(HORA_TXT)+'\nMOT: '+str(MOT_MATRICULA)+'\nCOB: '+str(COB_MATRICULA))
            notepad3.write('\n'+ str(err) +'\n')
            notepad3.write('\n=============================================================\n')        
        
        wtexto = 'Erro ao tentar na funcao bil_001  =  '+str(err)
        #insert_bil_erro(nome_txt,linha, wtexto)
        pass
    else:
        conn.commit()
        cursor.close()        

def insert_bil002(ID_PRINCIPAL, ID_BIL_002,TIPO, VERSAO, DATA_ARQ, HORA_ARQ, HORA_TXT, COD_EMISSOR_APLICACAO, 
                  ID_DESENHO_CARTAO, NR_SERIE, DIG_VERIFICADOR, NUM_INTERNO_CARTAO, CODIGO_APLICACAO, ID_EMISSOR,
                  TSN_CARTAO, VALOR_TARIFA, VALOR_TARIFA_ANT, VALOR_DEBITADO, DIF_VALOR_DEBITADO, VALOR_PROMOCAO,
                  VALOR_ACUMULADO, SECAO_ENTRADA, SECAO_SAIDA, STATUS, NR_ONIBUS, MOT_TSP, MOT_COD_PRODATA, 
                  MOT_MATRICULA, COB_TSP, COB_COD_PRODATA, COB_MATRICULA, TIPO_EMBARQUE, PROVIDER_ID, EXT_USER_CTR, 
                  EXT_VAL_DATE, ID_TIPO_DEBITO, ID_MENSAGEM_DEBITO, TP_CREDIT_PURSE_A, TP_CREDIT_PURSE_B, CSN_PURSE_A, 
                  CSN_PURSE_B, NUMERO_CHIP_SAM, LSEQUENCE_NUMBER, AVL_STATUS, ASS_REG, TSN_REPETIDO, NR_CARTAO_TXT, 
                  NOME_APLICACAO,COD_APP_002,COD_EMPRESA, NOME_EMPRESA, NR_LINHA, NOME_LINHA, TURNO, DATA_OPERACIONAL,
                  EMISSOR, APLICACAO):
    try:
        cursor = conn.cursor()
        sql_insert = """INSERT INTO BIL_002 (ID_PRINCIPAL, ID_BIL_002, TIPO, VERSAO, DATA_ARQ, HORA_ARQ, HORA_TXT, 
                                             COD_EMISSOR_APLICACAO, ID_DESENHO_CARTAO, NR_SERIE, DIG_VERIFICADOR, 
                                             NUM_INTERNO_CARTAO, CODIGO_APLICACAO, ID_EMISSOR, TSN_CARTAO, VALOR_TARIFA,
                                             VALOR_TARIFA_ANT, VALOR_DEBITADO, DIF_VALOR_DEBITADO, VALOR_PROMOCAO, 
                                             VALOR_ACUMULADO, SECAO_ENTRADA, SECAO_SAIDA, STATUS, NR_ONIBUS, MOT_TSP, 
                                             MOT_COD_PRODATA, MOT_MATRICULA, COB_TSP, COB_COD_PRODATA, COB_MATRICULA, 
                                             TIPO_EMBARQUE, PROVIDER_ID, EXT_USER_CTR, EXT_VAL_DATE, ID_TIPO_DEBITO, 
                                             ID_MENSAGEM_DEBITO, TP_CREDIT_PURSE_A, TP_CREDIT_PURSE_B, CSN_PURSE_A,
                                             CSN_PURSE_B, NUMERO_CHIP_SAM, LSEQUENCE_NUMBER, AVL_STATUS, ASS_REG,
                                             TSN_REPETIDO,NR_CARTAO_TXT, NOME_APLICACAO,COD_APP_002,COD_EMPRESA,
                                             NOME_EMPRESA, NR_LINHA, NOME_LINHA, TURNO, DATA_OPERACIONAL, EMISSOR,
                                             APLICACAO) 
                                             
                       VALUES(:1, :2, :3, :4, :5, :6, :7, :8, :9, :10, :11, :12, :13, :14, :15, :16, :17, :18, :19, :20, :21,
                              :22, :23, :24, :25, :26, :27, :28, :29, :30, :31, :32, :33, :34, :35, :36, :37, :38, :39, :40,
                              :41, :42, :43, :44, :45, :46, :47, :48, :49, :50, :51, :52, :53, :54, :55, :56, :57)"""
        
        cursor.execute(sql_insert, (ID_PRINCIPAL, ID_BIL_002, TIPO, VERSAO, DATA_ARQ, HORA_ARQ, HORA_TXT, 
                                    COD_EMISSOR_APLICACAO, ID_DESENHO_CARTAO, NR_SERIE, DIG_VERIFICADOR, 
                                    NUM_INTERNO_CARTAO, CODIGO_APLICACAO, ID_EMISSOR, TSN_CARTAO, VALOR_TARIFA, 
                                    VALOR_TARIFA_ANT, VALOR_DEBITADO, DIF_VALOR_DEBITADO, VALOR_PROMOCAO, VALOR_ACUMULADO, 
                                    SECAO_ENTRADA, SECAO_SAIDA, STATUS, NR_ONIBUS, MOT_TSP, MOT_COD_PRODATA, MOT_MATRICULA, 
                                    COB_TSP, COB_COD_PRODATA, COB_MATRICULA, TIPO_EMBARQUE, PROVIDER_ID, EXT_USER_CTR, 
                                    EXT_VAL_DATE, ID_TIPO_DEBITO, ID_MENSAGEM_DEBITO, TP_CREDIT_PURSE_A, TP_CREDIT_PURSE_B, 
                                    CSN_PURSE_A, CSN_PURSE_B, NUMERO_CHIP_SAM, LSEQUENCE_NUMBER, AVL_STATUS, ASS_REG, 
                                    TSN_REPETIDO, NR_CARTAO_TXT, NOME_APLICACAO,COD_APP_002,COD_EMPRESA, NOME_EMPRESA, 
                                    NR_LINHA, NOME_LINHA, TURNO, DATA_OPERACIONAL, EMISSOR, APLICACAO))  
    except Exception as err:
        wsem_erro = True
        with open(Path(wdir_permissao / wlog_txt), 'a') as notepad3:
            notepad3.write('Erro no insert_bil002\n')
            notepad3.write('NOME_ARQ:'+str(nome_txt)+'\nID PRINCIPAL: '+str(ID_PRINCIPAL)+'\nDATA: '+str(DATA_ARQ)+
                           '\nHORA: '+str(HORA_TXT)+'\nMOT: '+str(MOT_MATRICULA)+'\nCOB: '+str(COB_MATRICULA)+
                           '\nSEQ: '+str(ID_BIL_002)+'\nVALOR_TARIFA: '+str(VALOR_TARIFA)+
                           '\nVALOR_DEBITADO: '+str(VALOR_DEBITADO)+'\nSTATUS: '+str(STATUS)+'\nTSN: '+str(TSN_CARTAO))
            notepad3.write('\n'+ str(err) +'\n')
            notepad3.write('\n=============================================================\n')   
        
        shutil.move(arquivo, (caminho_arq / Path(dir_leitura[2]) / arquivo.name))
        
        #wtexto = 'Erro ao tentar na funcao bil_002  =  '+str(err)
        #insert_bil_erro(nome_txt,linha, wtexto)
        pass
    else:
        conn.commit()
        cursor.close() 
              
        
def insert_bil003(ID_PRINCIPAL, ID_BIL_003, TIPO, VERSAO, CODIGO_APLICACAO, COD_EMISSOR_APLICACAO, QTD_ROLETADAS, QTD_VALOR,
                  NR_ONIBUS, MOT_TSP, MOT_COD_PRODATA, MOT_MATRICULA, COB_TSP, COB_COD_PRODATA, COB_MATRICULA, DATA_ARQ, 
                  HORA_ARQ, HORA_TXT, ASS_REG):
    try:
        cursor = conn.cursor()
        sql_insert = """INSERT INTO BIL_003 (ID_PRINCIPAL, ID_BIL_003, TIPO, VERSAO, CODIGO_APLICACAO, COD_EMISSOR_APLICACAO,
                                             QTD_ROLETADAS, QTD_VALOR, NR_ONIBUS, MOT_TSP, MOT_COD_PRODATA, MOT_MATRICULA, 
                                             COB_TSP, COB_COD_PRODATA, COB_MATRICULA, DATA_ARQ, HORA_ARQ, HORA_TXT, ASS_REG) 
                       VALUES(:1, :2, :3, :4, :5, :6, :7, :8, :9, :10, :11, :12, :13, :14, :15, :16, :17, :18, :19)"""
        
        cursor.execute(sql_insert, (ID_PRINCIPAL, ID_BIL_003, TIPO, VERSAO, CODIGO_APLICACAO, COD_EMISSOR_APLICACAO,
                                    QTD_ROLETADAS, QTD_VALOR, NR_ONIBUS, MOT_TSP, MOT_COD_PRODATA, MOT_MATRICULA, 
                                    COB_TSP, COB_COD_PRODATA, COB_MATRICULA, DATA_ARQ, HORA_ARQ, HORA_TXT, ASS_REG))  
    except Exception as err:
        wtexto = 'Erro ao tentar na funcao bil_003  =  '+str(err)
        #insert_bil_erro(nome_txt,linha, wtexto)
        pass
    else:
        conn.commit()
        cursor.close()    
        
def insert_bil004(ID_PRINCIPAL, ID_BIL_004, TIPO, VERSAO, SWITCH, DATA_ARQ, HORA_ARQ, HORA_TXT, ASS_REG):
    try:
        cursor = conn.cursor()
        sql_insert = """INSERT INTO BIL_004 (ID_PRINCIPAL, ID_BIL_004, TIPO, VERSAO, SWITCH, DATA_ARQ, HORA_ARQ, HORA_TXT, 
                                             ASS_REG) 
                       VALUES(:1, :2, :3, :4, :5, :6, :7, :8, :9)"""
        cursor.execute(sql_insert, (ID_PRINCIPAL, ID_BIL_004, TIPO, VERSAO, SWITCH, DATA_ARQ, HORA_ARQ, HORA_TXT, ASS_REG))
    except Exception as err:
        wtexto = 'Erro ao tentar na funcao bil_004  =  '+str(err)
        #insert_bil_erro(nome_txt,linha, wtexto)
        pass
    else:
        conn.commit()
        cursor.close()

def insert_bil_erro(NOME_ARQUIVO, LINHA_AQUIVO, DESC_ERRO):
    try:
        wdata_hora = datetime.now()
        wdata_hora = wdata_hora.strftime("%d/%b/%Y %H:%M:%S")
        
        cursor = conn.cursor()
        sql_insert = """INSERT INTO BIL_ERRO(NOME_ARQUIVO, LINHA_AQUIVO, DESC_ERRO, DATA_HORA) 
                        VALUES(:1, :2, :3, :4)"""
        cursor.execute(sql_insert, (NOME_ARQUIVO, LINHA_AQUIVO, DESC_ERRO, wdata_hora))  
    except Exception as err:
        wtexto = 'Erro ao tentar na funcao bil_erro  =  '+str(err)
        pass
    else:
        conn.commit()
        cursor.close()   
        
        

def  insert_bil_painel(ID, EMPRESA, DATA, CARRO, MOT_BIL, COB_BIL, PEGADA_BIL, PEGADA_TXT_BIL,COD_EMPRESA_BIL, 
                       NOME_EMPRESA_BIL, NR_LINHA_BIL, NOME_LINHA_BIL, TURNO_BIL, NOME_ARQUIVO):
    try:
        cursor = conn.cursor()
        sql_insert = """INSERT INTO BIL_PAINEL (ID, EMPRESA, DATA, CARRO, MOT_BIL, COB_BIL, PEGADA_BIL, PEGADA_TXT_BIL,
                        COD_EMPRESA_BIL, NOME_EMPRESA_BIL, NR_LINHA_BIL, NOME_LINHA_BIL, TURNO_BIL, NOME_ARQUIVO) 
                        VALUES(:1, :2, :3, :4, :5, :6, :7, :8, :9, :10, :11, :12, :13, :14)"""
        cursor.execute(sql_insert, (ID, EMPRESA, DATA, CARRO, MOT_BIL, COB_BIL, PEGADA_BIL, PEGADA_TXT_BIL,
                                    COD_EMPRESA_BIL, NOME_EMPRESA_BIL, NR_LINHA_BIL, NOME_LINHA_BIL, TURNO_BIL, NOME_ARQUIVO))
    except Exception as err:
        wsem_erro = True
        with open(Path(wdir_permissao / wlog_txt), 'a') as notepad3:
            notepad3.write('Erro no insert_bil_painel\n')
            notepad3.write('NOME_ARQ:'+str(nome_txt)+'\nID PRINCIPAL: '+str(ID)+'\nCARRO: '+str(CARRO)+
                            '\nMOT_BIL: '+str(MOT_BIL)+'\nCOB_BIL: '+str(COB_BIL)+'\nPEGADA_TXT_BIL: '+str(PEGADA_TXT_BIL))
            notepad3.write('\n'+ str(err) +'\n')
            notepad3.write('\n=============================================================\n')                
        wtexto = 'Erro ao tentar na funcao bil_painel  =  '+str(err)
        #insert_bil_erro(nome_txt,linha, wtexto)
        pass
    else:
        conn.commit()
        cursor.close()
        
        
        
def importa_dados():
    global wlog_txt,wdir_permissao, wsem_erro, wtempo_inicial, wtempo_final, wcount_arquivos 
    global wtsn_rept, wtem_reg_002, wdir_permissao, linha, wlista_arquivo, wsair, nome_txt, wid_001, wid_002, wid_003, wid_004, \
    wid_005, wid_020, wid_021, wid_022, wid_023, wid_100, wid_principal, wcarro, wdata_arquivo, wnrinterno_002_ant, \
    wvalor_debitado_ant, wtsn_002_ant, wusado, wid_principal_res, wempresa_res, wdata_res, wnr_carro_res, wmot_tsp_res, \
    wmot_cod_prodata_res, wmot_matricula_res, wcob_tsp_res, wcob_cod_prodata_res, wcob_matricula_res, wqtd_pax_total_res, \
    wqtd_pax_grat_res, wqtd_pax_pagt_res, wqtd_pax_botao_res, wqtd_pax_vt_res, wqtd_pax_bu_res, wrec_bruta_res, \
    wrec_botao_res, wrec_vt_res, wrec_bu_bruto_res, wrec_bu_receb_res, wrec_subsidio_res,wcod_empresa, wnr_linha, \
    wnome_linha, wnome_empresa, wturno_princ, wmot_ant, wcob_ant, wnome_arquivo_db, wid_001_db, wcartoes, dir_leitura, arquivos, \
    wdata_util, wdesc_emissor, wdesc_aplicacao
    
    
    for linha in wlista_arquivo:
        if (caminho_arq / Path(dir_leitura[2]) / arquivo.name).exists():
            continue
            
        wusado = 'N'
        wsair = False
        tipo_reg = linha[:3]
        
        if(tipo_reg == '100'):
            wusado = 'S'
            wid_100 = wid_100 + 1
            wversao_100 = linha[3:6]
            wempresa_100 = linha[6:9]
            wtipo_onibus = linha[19:22]
            wseria_num_100 = linha[22:42]
            wlseqnumb_100 = linha[42:48]
            wassinat_100 = linha[48:52]
            wid_principal = prox_sequencia()
            w_nome_arq = nome_txt.split("_")
            wdata = w_nome_arq[5][6:8] +'/'+ w_nome_arq[5][4:6] +'/'+ w_nome_arq[5][:4]
            whora = w_nome_arq[5][8:]
            wdata_arquivo = atualiza_data(wdata, whora)
            wdata_malote = atualiza_data_dir(wdata, whora)
            wdata = mes_extenso(wdata)
            whora_txt = str(w_nome_arq[5][8:])
            wdata_util = date.today()
            wdata_util = wdata_util.strftime("%d/%b/%Y")
 
            
            wempresa = w_nome_arq[4]
            wseq_arq = w_nome_arq[3]
            ver = '02'
            wnome_arquivo_db = nome_txt
            if wempresa == '017':
                wcarro = '0'+ str(int(linha[9:19])) #w_nome_arq[1]
            else:
                wcarro = '1'+ str(int(linha[9:19])) #w_nome_arq[1]


            insert_principal(wid_principal, nome_txt, wdata, whora, ver, wcarro, wtipo_onibus, wempresa, wdata_util, wseq_arq, 
                             wdata_malote, whora_txt, wdata_arquivo, wnome_arquivo_db) 
            
            wcount_arquivos += 1
            
            if wsair == True:
                break
            else: 
                pass
            
            insert_bil100(wid_principal, wid_100, tipo_reg, wversao_100, wempresa_100, wcarro, wtipo_onibus, wseria_num_100,
                          wlseqnumb_100, wassinat_100)

        if(tipo_reg == '001'): 
            wusado = 'S'
            wversao_001 = 0  
            wempresa_001 = 0
            west_serv_001 = 0
            wdata_jul_001 = 0
            whora_001 = 0
            whora_texto_001 = 0
            wmot_tsp_001 = 0
            wmot_prodata_001 = 0
            wmatric_mot_001 = 0
            wcob_tsp_001 = 0
            wcob_prodata_001 = 0
            wmatric_cob_001 = 0
            wlinha_det_001 = 0
            wlinha_princ_001 = 0
            wcod_secao_001 = 0
            wsentido_001 = 0
            wturno_001 = 0
            wassinatura_001 = 0
           
            
            wid_001 += 1
            wversao_001  = linha[3:6]
            wempresa_001 = linha[6:9]
            west_serv_001 = linha[19:22]
            whora_001 = linha[27:32]
            whora_texto_001 = convert_hora(int(whora_001))
            wmot_tsp_001 = linha[32:35]
            wmot_prodata_001 = linha[35:45]
            wmatric_mot_001 = linha[45:55]
            
            
            wmatric_mot_temp = int(wmatric_mot_001)
            if str(wmatric_mot_temp) in wcartoes:
                wmatric_mot_001 = wcartoes[str(wmatric_mot_temp)]
                
                    
            wcob_tsp_001 = linha[55:58]
            wcob_prodata_001 = linha[58:68]
            wmatric_cob_001 = linha[68:78]
            
            wmatric_cob_temp = int(wmatric_cob_001)
            if str(wmatric_cob_temp) in wcartoes:
                wmatric_cob_001 = wcartoes[str(wmatric_cob_temp)]
        
   
            wlinha_det_001 = linha[78:83]
            wlinha_princ_001 = linha[83:88]
            wcod_secao_001 = linha[88:91]
            wsentido_001 = linha[91:94]
            wturno_001 = linha[94:95]
            wassinatura_001 = linha[95:99]
            
            if(int(wlinha_det_001) > 0):
                wcod_empresa = busca_linha(int(wlinha_det_001),0)
                wnr_linha = busca_linha(int(wlinha_det_001),1)
                wnome_linha = busca_linha(int(wlinha_det_001),2)
                
                if(wcod_empresa == '0070101'):
                    wnome_empresa = 'RIO DOURO'
                elif(wcod_empresa == '1280101'):
                    wnome_empresa = 'FLORES'
                elif(wcod_empresa == '1330101'):
                    wnome_empresa = 'REAL RIO'   
                elif(wcod_empresa == '1670101'):
                    wnome_empresa = 'MAGELI'  
                elif(wcod_empresa == '1710101'):
                    wnome_empresa = 'BEIRA MAR'  
                else:
                    wnome_empresa = 'NAO CADASTRADA'
            else:
                wcod_empresa = 999999
                wnr_linha = 999999
                wnome_linha = 'NAO CADASTRADA'
            
            if int(wturno_001) > 0:
                if int(wturno_001) == 1:
                    wturno_princ = 1
                else:
                    wturno_princ = 2
                    
            
            insert_bil001(wid_principal, wid_001, tipo_reg, wversao_001, wempresa_001, wcarro, west_serv_001, wdata_arquivo, 
                          whora_001, whora_texto_001, wmot_tsp_001, wmot_prodata_001, wmatric_mot_001, wcob_tsp_001,
                          wcob_prodata_001, wmatric_cob_001, wlinha_det_001, wlinha_princ_001, wcod_secao_001, wsentido_001, 
                          wturno_001, wassinatura_001)
            
          

            if((int(wmatric_mot_001) > 0 and int(wmot_ant) > 0 and int(wmatric_mot_001) != int(wmot_ant)) or 
               (int(wmatric_cob_001) > 0 and int(wcob_ant) > 0 and int(wmatric_cob_001) != int(wcob_ant))):
                          
                if wtem_reg_002 == True:
                    wtem_reg_002 = False
                    wid_002 = 0
                    wnome_arquivo_db = nome_txt[:-9]+'99.' + str(wid_001_db) + '.txt'
                    wid_principal = prox_sequencia()

                    insert_principal(wid_principal, nome_txt, wdata, whora, ver, wcarro, wtipo_onibus, wempresa, wdata_util, wseq_arq, 
                                     wdata_malote, whora_txt, wdata_arquivo, wnome_arquivo_db) 

                    wid_001_db += 1
            

            if(int(wmatric_mot_001) > 0):
                if(int(west_serv_001) in (3,4,5)):
                    wmot_ant = wmatric_mot_001
                    
            if(int(wmatric_cob_001) > 0):
                if(int(west_serv_001) in (3,4,5)):
                    wcob_ant = wmatric_cob_001


                    
        if(tipo_reg == '002'):
            wusado = 'S'
            wid_002 += 1
            if(wid_002 == 1):
                wnrinterno_002_ant = 0
                wtsn_002_ant = 0
                wvalor_debitado_ant = 0
            wtsn_rept = 'N'    
            wversao_002  = linha[3:6]
            whora_002= 0
            whora_texto_002= 0
            wemissor_002= 0
            wdesenho_002= 0
            wserie_002= 0
            wsubsidio = 0
            wdigverif_002= 0
            wnrinterno_002= 0
            waplicacao_002= 0
            wemissor_app_002= 0
            wtsn_002= 0
            wtarifa_002= 0
            wtarifa_ant_002= 0
            wvalor_debit_002= 0
            wdif_valor_debit_002 = 0
            wvalor_desc_002= 0
            wvalor_acum_002= 0
            wsecao_ent_002= 0
            wsecao_saida_002= 0
            wmot_tsp_002= 0
            wmot_prodata_002= 0
            wmatricula_mot_002= 0
            wcob_tsp_002= 0
            wcob_prodata_002= 0
            wmatricula_cob_002= 0
            wtp_embarque_002= 0
            wprovider_id_002= 0
            wexit_use_ctr_002= 0
            wexit_val_date_002 = '01/01/1900'
            wtp_debito_002= 0
            wmsg_debito_002= 0
            wtp_purse_a= 0
            wtp_purse_b= 0
            wcsn_purse_a= 0
            wcsn_purse_b= 0
            wnr_chipsam_002= 0
            wlseqnumber_002= 0
            wavl_status_002= 0
            wassinatura_002 = 0
            wnovo_valor = 0
            wnr_cartao_txt = 'nao cadastrado'
            wnome_aplicacao = 'nao cadastrado'
            wcod_app = 0

            if(wversao_002 == '001'):
                whora_002 = linha[11:16]
                whora_texto_002 = convert_hora(int(whora_002))
                wdata_002 = converte_juliana_bil002(linha[6:11])
                wdata_operacional_002 = formata_data(wdata_002)
                wdata_002 = atualiza_data(wdata_002, whora_texto_002)
                wemissor_002 = linha[16:19]
                wdesenho_002 = linha[19:22]
                wserie_002 = linha[22:30]
                wdigverif_002 = linha[30:31]
                wnrinterno_002 = linha[31:51]
                waplicacao_002 = linha[51:55]
                wemissor_app_002 = linha[55:58]
                wtsn_002 = linha[58:63]
                wtarifa_002 = linha[63:71]
                wtarifa_ant_002 = linha[71:77]
                wvalor_debit_002 = linha[77:83]
                wstatus_002 = linha[83:84]
                wmot_tsp_002 = linha[94:97]
                wmot_prodata_002 = linha[97:107]
                wmatricula_mot_002 = linha[107:117]
                wcob_tsp_002 = linha[117:120]
                wcob_prodata_002 = linha[120:130]
                wmatricula_cob_002 = linha[130:140]
                wnr_chipsam_002 = linha[140:160]
                wlseqnumber_002 = linha[160:166]
                wavl_status_002 = linha[166:168]
                wassinatura_002 = linha[168:172]

            elif(wversao_002 == '002'):
                whora_002 = linha[11:16]
                whora_texto_002 = str(convert_hora(int(whora_002)))
                wdata_002 = converte_juliana_bil002(linha[6:11])
                wdata_operacional_002 = formata_data(wdata_002)
                wdata_002 = atualiza_data(wdata_002, whora_texto_002)               
                wemissor_002 = linha[16:19]
                wdesenho_002 = linha[19:21]
                wserie_002 = linha[21:29]
                wdigverif_002 = linha[29:30]
                wnrinterno_002 = linha[30:50]
                waplicacao_002 = linha[50:54]
                wemissor_app_002 = linha[54:57]
                wtsn_002 = linha[57:62]
                wtarifa_002 = linha[62:68]
                wtarifa_ant_002 = linha[68:74]
                wvalor_debit_002 = linha[74:80]
                wsecao_ent_002 = linha[80:88]
                wsecao_saida_002 = linha[88:96]
                wstatus_002 = linha[96:97]
                wmot_tsp_002 = linha[107:110]
                wmot_prodata_002 = linha[110:120]
                wmatricula_mot_002 = linha[120:130]
                wcob_tsp_002 = linha[130:133]
                wcob_prodata_002 = linha[133:143]
                wmatricula_cob_002 = linha[143:153]
                wtp_embarque_002 = linha[153:154]
                wnr_chipsam_002 = linha[154:174]
                wlseqnumber_002 = linha[174:180]
                wassinatura_002 = linha[180:184]

            elif(wversao_002 == '003'):
                whora_002 = linha[11:16]
                whora_texto_002 = convert_hora(int(whora_002))
                wdata_002 = converte_juliana_bil002(linha[6:11])
                wdata_operacional_002 = formata_data(wdata_002)
                wdata_002 = atualiza_data(wdata_002, whora_texto_002)                
                wemissor_002 = linha[16:19]
                wdesenho_002 = linha[19:21]
                wserie_002 = linha[21:29]
                wdigverif_002 = linha[29:30]
                wnrinterno_002 = linha[30:50]
                waplicacao_002 = linha[50:54]
                wemissor_app_002 = linha[54:57]
                wtsn_002 = linha[57:62]
                wtarifa_002 = linha[62:68]
                wtarifa_ant_002 = linha[68:74]
                wvalor_debit_002 = linha[74:80]
                wsecao_ent_002 = linha[80:88]
                wsecao_saida_002 = linha[88:96]
                wstatus_002 = linha[96:97]
                wmot_tsp_002 = linha[107:110]
                wmot_prodata_002 = linha[110:120]
                wmatricula_mot_002 = linha[120:130]
                wcob_tsp_002 = linha[130:133]
                wcob_prodata_002 = linha[133:143]
                wmatricula_cob_002 = linha[143:153]
                wtp_embarque_002 = linha[153:154]
                wprovider_id_002 = linha[154:174]
                wnr_chipsam_002 = linha[174:194]
                wlseqnumber_002 = linha[194:200]
                wavl_status_002 = linha[200:202]
                wassinatura_002 = linha[202:206]   

            elif(wversao_002 == '004'):
                whora_002 = linha[11:16]
                whora_texto_002 = convert_hora(int(whora_002))
                wdata_002 = converte_juliana_bil002(linha[6:11])
                wdata_operacional_002 = formata_data(wdata_002)
                wdata_002 = atualiza_data(wdata_002, whora_texto_002)                
                wemissor_002 = linha[16:19]
                wdesenho_002 = linha[19:21]
                wserie_002 = linha[21:29]
                wdigverif_002 = linha[29:30]
                wnrinterno_002 = linha[30:50]
                waplicacao_002 = linha[50:54]
                wemissor_app_002 = linha[54:57]
                wtsn_002 = linha[57:62]
                wtarifa_002 = linha[62:68]
                wtarifa_ant_002 = linha[68:74]
                wvalor_debit_002 = linha[74:80]
                wsecao_ent_002 = linha[80:88]
                wsecao_saida_002 = linha[88:96]
                wstatus_002 = linha[96:97]
                wmot_tsp_002 = linha[107:110]
                wmot_prodata_002 = linha[110:120]
                wmatricula_mot_002 = linha[120:130]
                wcob_tsp_002 = linha[130:133]
                wcob_prodata_002 = linha[133:143]
                wmatricula_cob_002 = linha[143:153]
                wtp_embarque_002 = linha[153:154]
                wprovider_id_002 = linha[154:174]
                wexit_use_ctr_002 = linha[174:179]
                wexit_val_date_002 = converte_juliana(linha[179:184])
                wnr_chipsam_002 = linha[184:204]
                wlseqnumber_002 = linha[204:210]
                wavl_status_002 = linha[210:212]
                wassinatura_002 = linha[212:216]  

            elif(wversao_002 == '005'):
                whora_002 = linha[11:16]
                whora_texto_002 = convert_hora(int(whora_002))
                wdata_002 = converte_juliana_bil002(linha[6:11])
                wdata_operacional_002 = formata_data(wdata_002)
                wdata_002 = atualiza_data(wdata_002, whora_texto_002)                
                wemissor_002 = linha[16:19]
                wdesenho_002 = linha[19:21]
                wserie_002 = linha[21:29]
                wdigverif_002 = linha[29:30]
                wnrinterno_002 = linha[30:50]
                waplicacao_002 = linha[50:54]
                wemissor_app_002 = linha[54:57]
                wtsn_002 = linha[57:62]
                wtarifa_002 = linha[62:68]
                wtarifa_ant_002 = linha[68:74]
                wvalor_debit_002 = linha[74:80]
                wvalor_desc_002 = linha[80:86]
                wvalor_acum_002 = linha[86:92]
                wsecao_ent_002 = linha[92:100]
                wsecao_saida_002 = linha[100:108]
                wstatus_002 = linha[108:109]
                wmot_tsp_002 = linha[119:122]
                wmot_prodata_002 = linha[122:132]
                wmatricula_mot_002 = linha[132:142]
                wcob_tsp_002 = linha[142:145]
                wcob_prodata_002 = linha[145:155]
                wmatricula_cob_002 = linha[155:165]
                wtp_embarque_002 = linha[165:166]
                wtp_debito_002 = linha[166:167]
                wmsg_debito_002 = linha[167:180]
                wnr_chipsam_002 = linha[180:200]
                wlseqnumber_002 = linha[200:206]
                wassinatura_002 = linha[206:210]    

            elif(wversao_002 == '006'):
                whora_002 = linha[11:16]
                whora_texto_002 = convert_hora(int(whora_002))
                wdata_002 = converte_juliana_bil002(linha[6:11])
                wdata_operacional_002 = formata_data(wdata_002)
                wdata_002 = atualiza_data(wdata_002, whora_texto_002)                
                wemissor_002 = linha[16:19]
                wdesenho_002 = linha[19:21]
                wserie_002 = linha[21:29]
                wdigverif_002 = linha[29:30]
                wnrinterno_002 = linha[30:50]
                wemissor_app_002 = linha[50:54]
                wemissor_002 = linha[54:57]
                wtsn_002 = linha[57:62]
                wtarifa_002 = linha[62:68]
                wtarifa_ant_002 = linha[68:74]
                wvalor_debit_002 = linha[74:80]
                wdif_valor_debit_002 = linha[80:86]
                wsecao_ent_002 = linha[86:92]
                wsecao_saida_002 = linha[92:100]
                wstatus_002 = linha[100:101]
                wmot_tsp_002 = linha[111:114]
                wmot_prodata_002 = linha[114:124]
                wmatricula_mot_002 = linha[124:134]
                wcob_tsp_002 = linha[134:136]
                wcob_prodata_002 = linha[136:146]
                wmatricula_cob_002 = linha[146:156]
                wtp_embarque_002 = linha[156:157]
                wnr_chipsam_002 = linha[157:177]
                wlseqnumber_002 = linha[177:197]
                wavl_status_002 = linha[197:199]
                wassinatura_002 = linha[199:203]

            elif(wversao_002 == '007'):
                whora_002 = linha[11:16]
                whora_texto_002 = convert_hora(int(whora_002))
                wdata_002 = converte_juliana_bil002(linha[6:11])
                wdata_operacional_002 = formata_data(wdata_002)
                wdata_002 = atualiza_data(wdata_002, whora_texto_002)                
                wemissor_002 = linha[16:19]
                wdesenho_002 = linha[19:21]
                wserie_002 = linha[21:29]
                wdigverif_002 = linha[29:30]
                wnrinterno_002 = linha[30:50]
                waplicacao_002 = linha[50:54]
                wemissor_app_002 = linha[54:57]
                wtsn_002 = linha[57:62]
                wtarifa_002 = linha[62:68]
                wtarifa_ant_002 = linha[68:74]
                wvalor_debit_002 = linha[74:80]
                wstatus_002 = linha[80:81]
                wmot_tsp_002 = linha[91:94]
                wmot_prodata_002 = linha[94:104]
                wmatricula_mot_002 = linha[104:114]
                wcob_tsp_002 = linha[114:117]
                wcob_prodata_002 = linha[117:127]
                wmatricula_cob_002 = linha[127:137]
                wtp_embarque_002 = linha[137:138]
                wnr_chipsam_002 = linha[138:158]
                wlseqnumber_002 = linha[158:164]
                wassinatura_002 = linha[164:168]

            elif(wversao_002 == '008'):
                whora_002 = linha[11:16]
                whora_texto_002 = convert_hora(int(whora_002))
                wdata_002 = converte_juliana_bil002(linha[6:11])
                wdata_operacional_002 = formata_data(wdata_002)
                wdata_002 = atualiza_data(wdata_002, whora_texto_002)                
                wemissor_002 = linha[16:19]
                wdesenho_002 = linha[19:21]
                wserie_002 = linha[21:29]
                wdigverif_002 = linha[29:30]
                wnrinterno_002 = linha[30:50]
                waplicacao_002 = linha[50:54]
                wemissor_app_002 = linha[54:57]
                wtsn_002 = linha[57:62]
                wtarifa_002 = linha[62:68]
                wtarifa_ant_002 = linha[68:74]
                wvalor_debit_002 = linha[74:80]
                wstatus_002 = linha[80:81]
                wmot_tsp_002 = linha[91:94]
                wmot_prodata_002 = linha[94:104]
                wmatricula_mot_002 = linha[104:114]
                wcob_tsp_002 = linha[114:117]
                wcob_prodata_002 = linha[117:127]
                wmatricula_cob_002 = linha[127:137]
                wtp_embarque_002 = linha[137:138]
                wprovider_id_002 = linha[138:158]
                wnr_chipsam_002 = linha[158:178]
                wlseqnumber_002 = linha[178:184]
                wavl_status_002 = linha[184:186]
                wassinatura_002 = linha[186:190]

            elif(wversao_002 == '009'): #Não implementada
                whora_002 = linha[11:16]
                whora_texto_002 = convert_hora(int(whora_002))
                wdata_002 = converte_juliana_bil002(linha[6:11])
                wdata_operacional_002 = formata_data(wdata_002)
                wdata_002 = atualiza_data(wdata_002, whora_texto_002)                
                wemissor_002 = linha[16:19]
                wdesenho_002 = linha[19:21]
                wserie_002 = linha[21:29]
                wdigverif_002 = linha[29:30]
                wnrinterno_002 = linha[30:50]
                waplicacao_002 = linha[50:54]
                wemissor_app_002 = linha[54:57]
                wtsn_002 = linha[57:62]
                wvalor_debit_002 = linha[62:68]
                wsecao_ent_002 = linha[68:76]
                wsecao_saida_002 = linha[76:84]
                wstatus_002 = linha[84:85]
                wmot_tsp_002 = linha[95:98]
                wmot_prodata_002 = linha[98:108]
                wmatricula_mot_002 = linha[108:118]
                wcob_tsp_002 = linha[118:121]
                wcob_prodata_002 = linha[121:131]
                wmatricula_cob_002 = linha[131:141]
                wtp_embarque_002 = linha[141:142]
                wprovider_id_002 = linha[142:162]
                wassinatura_002 = linha[162:166]


            elif(wversao_002 == '011'):
                whora_002 = linha[11:16]
                whora_texto_002 = convert_hora(int(whora_002))
                wdata_002 = converte_juliana_bil002(linha[6:11])
                wdata_operacional_002 = formata_data(wdata_002)
                wdata_002 = atualiza_data(wdata_002, whora_texto_002)                
                wemissor_002 = linha[16:19]
                wdesenho_002 = linha[19:21]
                wserie_002 = linha[21:29]
                wdigverif_002 = linha[29:30]
                wnrinterno_002 = linha[30:50]
                waplicacao_002 = linha[50:54]
                wemissor_app_002 = linha[54:57]
                wtsn_002 = linha[57:62]
                wtarifa_002 = linha[62:68]
                wtarifa_ant_002 = linha[68:74]
                wvalor_debit_002 = linha[74:80]
                wvalor_desc_002 = linha[80:86]
                wvalor_acum_002 = linha[86:92]
                wsecao_ent_002 = linha[92:100]
                wsecao_saida_002 = linha[100:108]
                wstatus_002 = linha[108:109]
                wmot_tsp_002 = linha[119:122]
                wmot_prodata_002 = linha[122:132]
                wmatricula_mot_002 = linha[132:142]
                wcob_tsp_002 = linha[142:145]
                wcob_prodata_002 = linha[145:155]
                wmatricula_cob_002 = linha[155:165]
                wtp_embarque_002 = linha[165:166]
                wprovider_id_002 = linha[166:186]
                wtp_debito_002 = linha[186:187]
                wmsg_debito_002 = linha[187:190]
                wtp_purse_a = linha[190:192]
                wtp_purse_b = linha[192:194]
                wcsn_purse_a = linha[194:204]
                wcsn_purse_b = linha[204:214]
                wnr_chipsam_002 = linha[214:234]
                wlseqnumber_002 = linha[234:240]
                wavl_status_002 = linha[240:242]
                wassinatura_002 = linha[242:246]



            if(int(wnrinterno_002) == int(wnrinterno_002_ant)):
                if(int(wtsn_002) == int(wtsn_002_ant) and int(wtsn_002_ant) > 0):
                    wtsn_rept = 'S'
                    wid_002 -= 1
                    wnovo_valor = (float(wvalor_debitado_ant)/100) + (float(wvalor_debit_002)/100)
                    
                    if wtp_debito_002 == '3':
                        if float(wtarifa_ant_002) > 0 and float(wtarifa_002) > float(wtarifa_ant_002):
                            wsubsidio = round(float(wtarifa_ant_002)/100 - wnovo_valor,2)
                            wnome_aplicacao = 'BU ESTADUAL'
                            wcod_app = 3
                        else:
                            wsubsidio = round(float(wtarifa_ant_002)/100 - wnovo_valor,2)
                            wnome_aplicacao = 'BU ESTADUAL'
                            wcod_app = 3
                    update_tsn_repetido(wnovo_valor, wsubsidio, wid_principal, wnrinterno_002, wtsn_002)
            else:
                wtsn_rept = 'N'

                
            if wtp_debito_002 == '3' and wnome_aplicacao == 'nao cadastrado':
                if float(wtarifa_ant_002) > 0 and float(wtarifa_002) > float(wtarifa_ant_002):
                    if int(wstatus_002) == 0:    
                        wsubsidio = round(float(wtarifa_ant_002)/100 - (float(wvalor_debit_002)/100),2) 
                        if(wtsn_rept == 'N'):
                            wrec_bu_receb_res += round((float(wvalor_debit_002)/100),2)
                            wrec_bu_bruto_res += float(wtarifa_ant_002)/100
                            wrec_bruta_res +=  float(wtarifa_ant_002)/100
                            wtarifa_002 = (float(wtarifa_ant_002)/100)
                    wnome_aplicacao = 'BU ESTADUAL'
                    wcod_app = 3
                    
                else:
                    wsubsidio = round(float(wtarifa_002)/100 - (float(wvalor_debit_002)/100),2)
                    if int(wstatus_002) == 0: 
                        wrec_bu_receb_res += round((float(wvalor_debit_002)/100),2)
                        wrec_bu_bruto_res += float(wtarifa_002)/100
                        wrec_bruta_res +=  float(wtarifa_002)/100
                        wtarifa_002 = (float(wtarifa_002)/100)
                    wnome_aplicacao = 'BU ESTADUAL'
                    wcod_app = 3
                if int(wstatus_002) == 0:
                    if(wtsn_rept == 'N'):
                        wrec_subsidio_res += wsubsidio    
                        wqtd_pax_bu_res += 1
                        wqtd_pax_pagt_res += 1
                        wqtd_pax_total_res += 1
                
            else:
                if float(wtarifa_ant_002) > 0 and float(wtarifa_002) > float(wtarifa_ant_002):
                    wtarifa_002 = (float(wtarifa_ant_002)/100)
                else:
                    wtarifa_002 = (float(wtarifa_002)/100)
                    
                wsubsidio = 0
                if waplicacao_002 is None:
                    waplicacao_002 = '0000'
                if wtsn_002 is None:
                    wtsn_002 = '00000'

                if int(waplicacao_002) < 500 and wtsn_002 == '00000':    
                    wnome_aplicacao = 'BOTAO'
                    wcod_app = 1
                    if int(wstatus_002) == 0:
                        wqtd_pax_botao_res += 1
                        wqtd_pax_pagt_res += 1
                        wqtd_pax_total_res += 1
                        wrec_botao_res += wtarifa_002
                        wrec_bruta_res +=  wtarifa_002 
                    
                elif int(waplicacao_002) < 500 and int(wtsn_002) > 0:
                    wnome_aplicacao = 'VT'
                    wcod_app = 2
                    if int(wstatus_002) == 0:
                        if(wtsn_rept == 'N'):
                            wqtd_pax_vt_res += 1
                            wqtd_pax_pagt_res += 1
                            wqtd_pax_total_res += 1
                            wrec_bruta_res +=  wtarifa_002  
                            wrec_vt_res += wtarifa_002
                else:
                    wnome_aplicacao = 'GRATUIDADE'
                    wcod_app = 4
                    wtarifa_002 = 0
                    if int(wstatus_002) == 0:
                        wqtd_pax_grat_res += 1
                        wqtd_pax_total_res += 1

            wvalor_debitado_ant = wvalor_debit_002  

            wtarifa_ant_002 = (float(wtarifa_ant_002)/100)
            wvalor_debit_002 = (float(wvalor_debit_002)/100)
            wvalor_desc_002 = (float(wvalor_desc_002)/100)
            wvalor_acum_002 = (float(wvalor_acum_002)/100)
            wnrinterno_002_ant = wnrinterno_002
            wtsn_002_ant = wtsn_002
            wnr_cartao_txt = wemissor_002+'.'+wdesenho_002+'.'+wserie_002+'-'+wdigverif_002
            
            wdata_arquivo = wdata_002
            
            wmatric_mot_temp = int(wmatricula_mot_002)
            if str(wmatric_mot_temp) in wcartoes:
                wmatricula_mot_002 = wcartoes[str(wmatric_mot_temp)]
                    
            wmatric_cob_temp = int(wmatricula_cob_002)
            if str(wmatric_cob_temp) in wcartoes:
                wmatricula_cob_002 = wcartoes[str(wmatric_cob_temp)]
            
            
            
            wdesc_emissor   = desc_emissor(int(wemissor_app_002))
            wdesc_aplicacao = desc_aplicacao(int(wemissor_app_002), int(waplicacao_002))
            
            
            if(wtsn_rept == 'N'):
                
                wturno_princ = 1
                
                insert_bil002(int(wid_principal), int(wid_002), int(tipo_reg), int(wversao_002), wdata_arquivo, int(whora_002), whora_texto_002, 
                              int(wemissor_002),int(wdesenho_002), int(wserie_002),int(wdigverif_002), int(wnrinterno_002), int(waplicacao_002), 
                              int(wemissor_app_002), wtsn_002, wtarifa_002, wtarifa_ant_002, wvalor_debit_002, wsubsidio,
                              wvalor_desc_002, wvalor_acum_002, int(wsecao_ent_002), int(wsecao_saida_002), int(wstatus_002), 
                              int(wcarro), wmot_tsp_002, int(wmot_prodata_002), int(wmatricula_mot_002), int(wcob_tsp_002), int(wcob_prodata_002),
                              int(wmatricula_cob_002), int(wtp_embarque_002), int(wprovider_id_002), int(wexit_use_ctr_002), wexit_val_date_002, 
                              int(wtp_debito_002), int(wmsg_debito_002), int(wtp_purse_a), int(wtp_purse_b), int(wcsn_purse_a), int(wcsn_purse_b), 
                              wnr_chipsam_002, int(wlseqnumber_002), int(wavl_status_002), int(wassinatura_002), wtsn_rept, wnr_cartao_txt, 
                              wnome_aplicacao, int(wcod_app), wcod_empresa, wnome_empresa, wnr_linha, wnome_linha, int(wturno_princ),
                              wdata_operacional_002, wdesc_emissor, wdesc_aplicacao)
                
            if(wtem_reg_002 == False):
                wtem_reg_002 = True
                insert_bil_painel(wid_principal, wempresa, wdata_operacional_002, wcarro, wmatricula_mot_002, wmatricula_cob_002, 
                                  whora_002, whora_texto_002, wcod_empresa, wnome_empresa, wnr_linha, wnome_linha, wturno_princ,
                                  nome_txt)
                
                atualiza_financeiro_principal('S', wid_principal)
                
                wid_principal_res = wid_principal
                wempresa_res = wempresa
                wdata_res = wdata_arquivo
                wnr_carro_res = wcarro
                wmot_tsp_res = wmot_tsp_002
                wmot_cod_prodata_res = wmot_prodata_002
                wmot_matricula_res = wmatricula_mot_002
                wcob_tsp_res = wcob_tsp_002
                wcob_cod_prodata_res = wcob_prodata_002
                wcob_matricula_res = wmatricula_cob_002
            

                
        if(tipo_reg == '003'):
            wusado = 'S'
            wid_003 += 1
            wversao_003  = linha[3:6]
            waplicacao_003 = 0
            wemissor_app_003 = 0
            wqtd_roletadas = 0
            wqtd_valor = 0
            wmot_tsp_003 = 0
            wmot_prodata_003 = 0
            wmatric_mot_003 = 0
            wcob_tsp_003 = 0
            wcob_prodata_003 = 0
            wmatric_cob_003 = 0
            whora_003 = 0
            whora_texto_003 = 0
            wassinatura_003 = 0

            waplicacao_003 = linha[6:10]
            wemissor_app_003 = linha[10:13]
            wqtd_roletadas = linha[13:18]
            wqtd_valor = linha[18:25]
            wmot_tsp_003 = linha[35:38]
            wmot_prodata_003 = linha[38:48]
            wmatric_mot_003 = linha[48:58]
            wcob_tsp_003 = linha[58:61]
            wcob_prodata_003 = linha[61:71]
            wmatric_cob_003 = linha[71:81]
            whora_003 = linha[86:91]
            whora_texto_003 = convert_hora(int(whora_003))
            wassinatura_003 = linha[91:95]

            wmatric_mot_temp = int(wmatric_mot_003)
            if str(wmatric_mot_temp) in wcartoes:
                wmatric_mot_003 = wcartoes[str(wmatric_mot_temp)]
                    
            wmatric_cob_temp = int(wmatric_cob_003)
            if str(wmatric_cob_temp) in wcartoes:
                wmatric_cob_003 = wcartoes[str(wmatric_cob_temp)]
            
            insert_bil003(wid_principal, wid_003, tipo_reg, wversao_003, waplicacao_003, wemissor_app_003, wqtd_roletadas, 
                          (float(wqtd_valor)/100), wcarro, wmot_tsp_003, wmot_prodata_003, wmatric_mot_003, wcob_tsp_003, 
                          wcob_prodata_003, wmatric_cob_003, wdata_arquivo, whora_003, whora_texto_003, wassinatura_003)

        if(tipo_reg == '004'):
            wusado = 'S'
            wid_004 += 1
            wversao_004  = linha[3:6]
            wswitch_004 = 0
            whora_004 = 0
            whora_texto_004 = 0
            assinatura_004 = 0
            if(wversao_004 == '010'):
                wswitch_004 = linha[6:8]
                whora_004 = linha[13:18]
                whora_texto_004 = convert_hora(int(whora_004))
                wemissor_004 = linha[18:21]
                wdesenho_004 = linha[21:23]
                wserie_004 = linha[23:31]
                wdigverif_004 = linha[31:32]
                wnrinterno_004 = linha[32:52]
                waplicacao_004 = linha[52:56]
                wemissor_app_004 = linha[56:59]
                assinatura_004 = linha[59:63]

                insert_bil004(wid_principal, wid_004, tipo_reg, wversao_004, wswitch_004, wdata_arquivo, whora_004, 
                              whora_texto_004, assinatura_004)
            else:
                wswitch_004 = linha[6:8]
                whora_004 = linha[13:18]
                whora_texto_004 = convert_hora(int(whora_004))
                assinatura_004 = linha[18:22]
                insert_bil004(wid_principal, wid_004, tipo_reg, wversao_004, wswitch_004, wdata_arquivo, whora_004, 
                              whora_texto_004, assinatura_004)

        if(wusado == 'N'):
            wnome_erro = 'Arquivo nao processado'
            #insert_bil_erro(nome_txt, linha, wnome_erro)

        if(linha == wlista_arquivo[-1] and wtem_reg_002 == False):
                atualiza_financeiro_principal('N', wid_principal)

In [6]:
wdir_permissao = Path('C:/Users/marcelo.francisco/Documents/')
#wdir_permissao = Path('/home/robo/Documents/')
wdat_log = 'LOG/LOG_EXECUCAO_IMPORTA_DADOS_BILHETAGEM_'
wlog_txt = 'LOG/LOG_EXECUCAO_IMPORTA_DADOS_BILHETAGEM_'


if Path(wdir_permissao / 'FLAG').exists():
    if Path(wdir_permissao / 'FLAG/FLAG_IMPORTA_DADOS_BILHETAGEM').exists():
        if Path(wdir_permissao / 'FLAG/FLAG_IMPORTA_DADOS_BILHETAGEM/FLAG.txt').exists():
            with open(Path(wdir_permissao / 'FLAG/FLAG_IMPORTA_DADOS_BILHETAGEM/FLAG.txt'), 'r') as notepad:
                wpermissao = notepad.read()
                if 'FALSE' in wpermissao:
                    if Path(wdir_permissao / 'LOG').exists():
                        wdat_log = wdat_log + str(datetime.today().strftime('%Y%m%d_%H%M')) + '.txt'
                        with open(Path(wdir_permissao / wdat_log), 'w') as notepad2:
                            notepad2.write('Execução Falhou em ' + str(datetime.today().strftime('%Y-%m-%d %H:%M')) \
                                            + ', Foi identificada a flag com valor False, o que pode significar que outra' + \
                                             ' instância do python está ainda em execução, ou a execução anterior falhou.')     
                    else:
                        Path(wdir_permissao / 'LOG').mkdir()
                        wdat_log = wdat_log + str(datetime.today().strftime('%Y%m%d_%H%M')) + '.txt'
                        with open(Path(wdir_permissao / wdat_log), 'w') as notepad2:
                            notepad2.write('Execução Falhou em ' + str(datetime.today().strftime('%Y-%m-%d %H:%M')) \
                                           + ', Foi identificada a flag com valor False, o que pode significar que outra' \
                                           + 'instância do python está ainda em execução, ou a execução anterior falhou.')
                            
                    with open(Path(wdir_permissao / 'FLAG/FLAG_IMPORTA_DADOS_BILHETAGEM/FLAG.txt'), 'w') as notepad:
                        notepad.write('TRUE')
                    sys.exit()
                        
                elif 'TRUE' in wpermissao:
                    with open(Path(wdir_permissao / 'FLAG/FLAG_IMPORTA_DADOS_BILHETAGEM/FLAG.txt'), 'w') as notepad:
                        notepad.write('FALSE')        
                    if Path(wdir_permissao / 'LOG').exists():
                        wsem_erro = False
                        wlog_txt = wlog_txt + str(datetime.today().strftime('%Y%m%d_%H%M')) + '.txt'
                        with open(Path(wdir_permissao / wlog_txt), 'a') as notepad3:
                            notepad3.write('=============================================================\n')
                            notepad3.write('                     INICIO DA EXECUCAO')
                            notepad3.write('\n=============================================================\n')     
                    else:
                        wsem_erro = False
                        Path(wdir_permissao / 'LOG').mkdir()
                        wlog_txt = wlog_txt + str(datetime.today().strftime('%Y%m%d_%H%M')) + '.txt'
                        with open(Path(wdir_permissao / wlog_txt), 'a') as notepad3:
                            notepad3.write('=============================================================\n')
                            notepad3.write('                     INICIO DA EXECUCAO')
                            notepad3.write('\n=============================================================\n')
        else:
            with open(Path(wdir_permissao / 'FLAG/FLAG_IMPORTA_DADOS_BILHETAGEM/FLAG.txt'), 'w') as notepad:
                notepad.write('FALSE')

            if Path(wdir_permissao / 'LOG').exists():
                wsem_erro = False
                wlog_txt = wlog_txt + str(datetime.today().strftime('%Y%m%d_%H%M')) + '.txt'
                with open(Path(wdir_permissao / wlog_txt), 'a') as notepad3:
                    notepad3.write('=============================================================\n')
                    notepad3.write('                     INICIO DA EXECUCAO')
                    notepad3.write('\n=============================================================\n')

            else:
                wsem_erro = False
                Path(wdir_permissao / 'LOG').mkdir()
                wlog_txt = wlog_txt + str(datetime.today().strftime('%Y%m%d_%H%M')) + '.txt'
                with open(Path(wdir_permissao / wlog_txt), 'a') as notepad3:
                    notepad3.write('=============================================================\n')
                    notepad3.write('                     INICIO DA EXECUCAO')
                    notepad3.write('\n=============================================================\n')
    else:
        Path(wdir_permissao / 'FLAG/FLAG_IMPORTA_DADOS_BILHETAGEM').mkdir()
        with open(Path(wdir_permissao / 'FLAG/FLAG_IMPORTA_DADOS_BILHETAGEM/FLAG.txt'), 'w') as notepad:
            notepad.write('FALSE')
        if Path(wdir_permissao / 'LOG').exists():
            wsem_erro = False
            wlog_txt = wlog_txt + str(datetime.today().strftime('%Y%m%d_%H%M')) \
                        + '.txt'
            with open(Path(wdir_permissao / wlog_txt), 'a') as notepad3:
                notepad3.write('=============================================================\n')
                notepad3.write('                     INICIO DA EXECUCAO')
                notepad3.write('\n=============================================================\n')
        else:
            Path(wdir_permissao / 'LOG').mkdir()
            wsem_erro = False
            wlog_txt = wlog_txt + str(datetime.today().strftime('%Y%m%d_%H%M')) \
                        + '.txt'
            with open(Path(wdir_permissao / wlog_txt), 'a') as notepad3:
                notepad3.write('=============================================================\n')
                notepad3.write('                     INICIO DA EXECUCAO')
                notepad3.write('\n=============================================================\n')
                    
else:
    Path(wdir_permissao / 'FLAG').mkdir()
    Path(wdir_permissao / 'FLAG/FLAG_IMPORTA_DADOS_BILHETAGEM').mkdir()
    with open(Path(wdir_permissao / 'FLAG/FLAG_IMPORTA_DADOS_BILHETAGEM/FLAG.txt'), 'w') as notepad:
        notepad.write('FALSE')   
    if Path(wdir_permissao / 'LOG').exists():
        wsem_erro = False
        wlog_txt = wlog_txt + str(datetime.today().strftime('%Y%m%d_%H%M')) + '.txt'
        with open(Path(wdir_permissao / wlog_txt), 'a') as notepad3:
            notepad3.write('=============================================================\n')
            notepad3.write('                     INICIO DA EXECUCAO')
            notepad3.write('\n=============================================================\n')
    else:
        wsem_erro = False
        Path(wdir_permissao / 'LOG').mkdir()
        wlog_txt = wlog_txt + str(datetime.today().strftime('%Y%m%d_%H%M')) + '.txt'
        with open(Path(wdir_permissao / wlog_txt), 'a') as notepad3:
            notepad3.write('=============================================================\n')
            notepad3.write('                     INICIO DA EXECUCAO')
            notepad3.write('\n=============================================================\n')
       
        
try: 
    #estabelece conexão
    conn = cx_Oracle.connect(user="news",
                             password="news",
                             dsn="192.168.0.3/tcp"
                            )
except Exception as err:
    print('Exceção ocorreu ao tentar estabalecer uma conexão com o banco de dados ', err)

        
dat_atual = date(2023, 10, 12)


#dat_atual = datetime.now().strftime('%d/%m/%Y')
#whora_dir = datetime.now().strftime('%H:%M:%S')
#dat_atual = atualiza_data_dir(dat_atual, whora_dir) 

cod_empresas = ['UD_006_' + f"{dat_atual:%Y%m%d}", 'UD_010_' + f"{dat_atual:%Y%m%d}", 'UD_017_' + f"{dat_atual:%Y%m%d}", \
                'UD_025_' + f"{dat_atual:%Y%m%d}", 'UD_027_' + f"{dat_atual:%Y%m%d}"]
dir_leitura = ['PROCESSANDO', 'BKP - BILHETAGEM', 'ERRO']

#caminho_arq = Path('/objetos/v3lan/IMPORTACAO/HG/' + 'UD_' + f"{dat_atual:%Y%m}" + '/' + 'UD_' + f"{dat_atual:%Y%m%d}")
#caminho_arq = Path('C:/Users/matheus.silveira/Desktop/HG/' + f"{dat_atual:%m}" + '/' + 'UD_' + f"{dat_atual:%Y%m%d}")
caminho_arq = Path('Y:/v3lan/IMPORTACAO/HG/' + 'UD_' + f"{dat_atual:%Y%m}" + '/' + 'UD_' + f"{dat_atual:%Y%m%d}")
for diretorios in dir_leitura:
    if not (caminho_arq / Path(diretorios)).exists():
        (caminho_arq / Path(diretorios)).mkdir()


for empresa in cod_empresas:
    if (caminho_arq / Path(empresa)).exists():
        arquivos = (caminho_arq / Path(empresa)).iterdir()
        for arquivo in arquivos:
            shutil.move(arquivo, (caminho_arq / Path(dir_leitura[0]) / arquivo.name))

arquivos = (caminho_arq / Path(dir_leitura[0])).iterdir()

for arquivo in arquivos:
    shutil.copy2(arquivo, (caminho_arq / Path(dir_leitura[1]) / arquivo.name))
    
    
arquivos = (caminho_arq / Path(dir_leitura[0])).iterdir()

wtem_reg_002 = False
wtempo_inicial = time.time()
wcount_arquivos = 0
wcartoes = cartao_provisorio()


try:
    for arquivo in arquivos: 
        nome_txt = arquivo.name
        #comando with fecha o arquivo automaticamante ao fim do processo
        with open((caminho_arq / Path(dir_leitura[0]) / nome_txt),'r') as dados_arquivo: 
            wlista_arquivo = dados_arquivo.readlines()

        #if (wtem_reg_002 == True): 
            #insert_resumo_carro(wid_principal_res, wempresa_res, wdata_res, wnr_carro_res, wmot_tsp_res, wmot_cod_prodata_res,
            #                    wmot_matricula_res, wcob_tsp_res, wcob_cod_prodata_res, wcob_matricula_res, wqtd_pax_total_res,   
            #                    wqtd_pax_grat_res, wqtd_pax_pagt_res, wqtd_pax_botao_res, wqtd_pax_vt_res, wqtd_pax_bu_res,      
            #                    round(wrec_bruta_res,2), round(wrec_botao_res,2), round(wrec_vt_res,2), 
            #                    round(wrec_bu_bruto_res,2), round(wrec_bu_receb_res,2), round(wrec_subsidio_res,2))

           # insert_resumo_empresa(wempresa_res, wdata_res, wqtd_pax_total_res, wqtd_pax_botao_res, wqtd_pax_vt_res,
             #                     wqtd_pax_bu_res, wqtd_pax_grat_res, round(wrec_bruta_res,2), round(wrec_botao_res,2), 
             #                     round(wrec_vt_res,2), round(wrec_bu_bruto_res,2),(round(wrec_bu_bruto_res,2) - round(wrec_subsidio_res,2)),
             #                     round(wrec_subsidio_res,2), 1)  


        wid_100 = 0
        wid_001 = 0
        wid_002 = 0
        wid_003 = 0
        wid_004 = 0
        wid_005 = 0
        wid_020 = 0
        wid_021 = 0
        wid_022 = 0
        wid_023 = 0
        wid_001_db = 90
        
        wid_principal_res = 0
        wempresa_res = 0
        wdata_res = 0
        wnr_carro_res = 0
        wmot_tsp_res = 0
        wmot_cod_prodata_res = 0
        wmot_matricula_res = 0
        wcob_tsp_res = 0
        wcob_cod_prodata_res = 0
        wcob_matricula_res = 0
        wqtd_pax_total_res = 0
        wqtd_pax_grat_res = 0
        wqtd_pax_pagt_res = 0
        wqtd_pax_botao_res = 0
        wqtd_pax_vt_res = 0
        wqtd_pax_bu_res = 0
        wrec_bruta_res = 0
        wrec_botao_res = 0
        wrec_vt_res = 0
        wrec_bu_bruto_res = 0
        wrec_bu_receb_res = 0
        wrec_subsidio_res = 0
        wtem_reg_002 = False
        wmot_ant = 0
        wcob_ant = 0
        wnome_arquivo_db = '0'
           
        importa_dados()
        
except Exception as err:
    print(err)
    wtempo_final = time.time()
    if wcount_arquivos == 0:
        tempo_medio = 'NENHUM ARQUIVO FOI IMPORTADO'
    else:
        tempo_medio = 'TEMPO MEDIO DE IMPORTACAO POR ARQUIVO: ' + str(int(wtempo_final - wtempo_inicial) / 
                                                                          wcount_arquivos) + ' segundos.'
        
    with open(Path(wdir_permissao / wlog_txt), 'a') as notepad3:
        notepad3.write('\n               ERRO DE EXECUCAO\n') 
        notepad3.write('\n' + str(err) + '\n')
        notepad3.write('\nTEMPO TOTAL DE EXECUCAOO: ' + convert_hora(int(wtempo_final - wtempo_inicial)) + '\n')
        notepad3.write('\nQUANTIDADE DE ARQUIVOS IMPORTADOS: ' + str(wcount_arquivos) + '\n')
        notepad3.write('\n' + tempo_medio + '\n')
        notepad3.write('\n=============================================================\n') 
    conn.close()
    
else:
    conn.close()
    
        
arquivos = (caminho_arq / Path(dir_leitura[0])).iterdir()
for arquivo in arquivos:
    os.remove(str(arquivo))       

wtempo_final = time.time()

if wsem_erro == False:
    if wcount_arquivos == 0:
        tempo_medio = 'NENHUM ARQUIVO FOI IMPORTADO'
    else:
        tempo_medio = 'TEMPO MEDIO DE IMPORTACAO POR ARQUIVO: ' + str(int(wtempo_final - wtempo_inicial) / 
                                                                          wcount_arquivos) + ' segundos.'
        
    with open(Path(wdir_permissao / wlog_txt), 'a') as notepad3:
        notepad3.write('               EXECUCAO CONCLUIDA COM SUCESSO\n')
        notepad3.write('\nTEMPO TOTAL DE EXECUCAO: ' + convert_hora(int(wtempo_final - wtempo_inicial)) + '\n')
        notepad3.write('\nQUANTIDADE DE ARQUIVOS IMPORTADOS: ' + str(wcount_arquivos) + '\n')
        notepad3.write('\n' + tempo_medio + '\n')
        notepad3.write('\n=============================================================\n')  
        
elif wsem_erro == True:
    if wcount_arquivos == 0:
        tempo_medio = 'NENHUM ARQUIVO FOI IMPORTADO'
    else:
        tempo_medio = 'TEMPO MEDIO DE IMPORTACAO POR ARQUIVO: ' + str(int(wtempo_final - wtempo_inicial) / 
                                                                          wcount_arquivos) + ' segundos.'
        
    with open(Path(wdir_permissao / wlog_txt), 'a') as notepad3:
        notepad3.write('\nTEMPO TOTAL DE EXECUCAO: ' + convert_hora(int(wtempo_final - wtempo_inicial)) + '\n')
        notepad3.write('\nQUANTIDADE DE ARQUIVOS IMPORTADOS: ' + str(wcount_arquivos) + '\n')
        notepad3.write('\n' + tempo_medio + '\n')
        notepad3.write('\n=============================================================\n') 

        
if Path(wdir_permissao / 'FLAG').exists():
    if Path(wdir_permissao / 'FLAG/FLAG_IMPORTA_DADOS_BILHETAGEM').exists():        
        with open(Path(wdir_permissao / 'FLAG/FLAG_IMPORTA_DADOS_BILHETAGEM/FLAG.txt'), 'w') as notepad:
            notepad.write('TRUE')
    else:
        Path(wdir_permissao / 'FLAG/FLAG_IMPORTA_DADOS_BILHETAGEM').mkdir()
        with open(Path(wdir_permissao / 'FLAG/FLAG_IMPORTA_DADOS_BILHETAGEM/FLAG.txt'), 'w') as notepad:
            notepad.write('TRUE')    
else:
    Path(wdir_permissao / 'FLAG').mkdir()
    Path(wdir_permissao / 'FLAG/FLAG_IMPORTA_DADOS_BILHETAGEM').mkdir()
    with open(Path(wdir_permissao / 'FLAG/FLAG_IMPORTA_DADOS_BILHETAGEM/FLAG.txt'), 'w') as notepad:
        notepad.write('TRUE')
        
print('PROCESSAMENTO EXECUTADO COM SUCESSO!!!')

PROCESSAMENTO EXECUTADO COM SUCESSO!!!
